# Data Cleaning and Feature Engineering

En aquest notebook es duran a terme tres processos: la neteja de dades i l'enginyeria de característiques. Aquests tres processos es realitzaran de forma conjunta, ja que aquesta és la forma natural de treballar durant l'exploració de dades. L'objectiu final és arribar a dos conjunts de dades finals nets i explorats que estiguin preparats per a la visualització de dades.


In [1]:
import os

import country_converter as coco
import geopandas as gpd

import numpy as np
import pandas as pd

from itertools import combinations
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

## Primer dataset: agri_country_year_df


In [2]:
agri_country_year_df = pd.read_csv('../data/visualization_input/agri_country_year_df.csv')
print("Primeres files del dataset:")
display(agri_country_year_df.head())
print("\nColumnes del dataset:")
print(agri_country_year_df.columns.tolist())


Primeres files del dataset:


,ISO3,Area Code,Area Code (M49),Area,Year Code,Year,Value_nutrient_3102_N_Agricultural_Use_t,Value_nutrient_3102_N_Use_per_area_of_cropland_kg/ha,Value_nutrient_3102_N_Use_per_capita_kg/cap,Value_nutrient_3102_N_Use_per_value_of_agricultural_production_g/Int$,...,urban_population_%,surface_temperature_change_celsius,agricultural_irrigated_land_%_of_total_agri_land,agricultural_land_%_total_land,arable_land_%_total_land,arable_land_ha_per_person,agriculture_forestry_fishing_value_added_%_GDP,agriculture_forestry_fishing_value_added_%_annual_growth,prec_mm_per_year,mean_annual_tas_deg_celsius
0,ARM,1,'051,Armenia,1992,1992,15000.0,31.06,4.20,NaN,...,64.462083,-1.281,NaN,41.095890,14.857745,0.118426,NaN,-8.700001,588.87,5.56
1,ARM,1,'051,Armenia,1993,1993,15000.0,30.43,4.34,NaN,...,64.865672,-0.931,NaN,41.482262,15.208992,0.125386,NaN,-5.799998,529.38,6.20
2,ARM,1,'051,Armenia,1994,1994,7000.0,14.00,2.08,NaN,...,64.736329,0.488,NaN,42.149631,15.279241,0.129287,NaN,3.099996,580.55,7.46
3,ARM,1,'051,Armenia,1995,1995,7000.0,14.17,2.12,NaN,...,64.255116,0.701,NaN,43.695118,15.279241,0.131516,NaN,4.000003,478.28,7.61
4,ARM,1,'051,Armenia,1996,1996,8000.0,15.81,2.44,NaN,...,63.563569,0.495,NaN,45.872849,15.806112,0.137248,NaN,1.994688,478.91,7.97



Columnes del dataset:
['ISO3', 'Area Code', 'Area Code (M49)', 'Area', 'Year Code', 'Year', 'Value_nutrient_3102_N_Agricultural_Use_t', 'Value_nutrient_3102_N_Use_per_area_of_cropland_kg/ha', 'Value_nutrient_3102_N_Use_per_capita_kg/cap', 'Value_nutrient_3102_N_Use_per_value_of_agricultural_production_g/Int$', 'Value_nutrient_3103_P2O5_Agricultural_Use_t', 'Value_nutrient_3103_P2O5_Use_per_area_of_cropland_kg/ha', 'Value_nutrient_3103_P2O5_Use_per_capita_kg/cap', 'Value_nutrient_3103_P2O5_Use_per_value_of_agricultural_production_g/Int$', 'Value_nutrient_3104_K2O_Agricultural_Use_t', 'Value_nutrient_3104_K2O_Use_per_area_of_cropland_kg/ha', 'Value_nutrient_3104_K2O_Use_per_capita_kg/cap', 'Value_nutrient_3104_K2O_Use_per_value_of_agricultural_production_g/Int$', 'Value_6601_land_area_ha', 'Value_6610_agricultural_land_ha', 'Value_6620_cropland_ha', 'Value_6621_arable_land_ha', 'Value_6650_permanent_crops_ha', 'Value_6655_permanent_meadows_and_pastures_ha', 'Value_pesticide_agricultural

El dataset conté tant Year Code com Year. Year Code és un identificador numèric que representa el mateix any que Year, per tant és redundant. Ens quedem només amb Year que és més llegible i directe per a l'anàlisi i visualització.


A més, el dataset conté múltiples columnes relacionades amb l'àrea geogràfica: ISO3, Area Code, Area Code (M49) i Area. ISO3 és el codi de tres lletres estàndard que identifica únicament cada país. Area Code i Area Code (M49) són codis numèrics redundants que no aporten informació addicional per a la visualització. Area conté el nom del país que és necessari per a la visualització. Per tant, ens quedem amb ISO3 per a la identificació única i Area per al nom llegible, i eliminem les columnes de codis numèrics redundants.


In [3]:
agri_country_year_df = agri_country_year_df.drop(columns=['Year Code', 'Area Code', 'Area Code (M49)'])
agri_country_year_df = agri_country_year_df.rename(columns={'Area': 'Country'})
print("Columnes després de netejar les columnes d'àrea:")
print(agri_country_year_df.columns.tolist())


Columnes després de netejar les columnes d'àrea:
['ISO3', 'Country', 'Year', 'Value_nutrient_3102_N_Agricultural_Use_t', 'Value_nutrient_3102_N_Use_per_area_of_cropland_kg/ha', 'Value_nutrient_3102_N_Use_per_capita_kg/cap', 'Value_nutrient_3102_N_Use_per_value_of_agricultural_production_g/Int$', 'Value_nutrient_3103_P2O5_Agricultural_Use_t', 'Value_nutrient_3103_P2O5_Use_per_area_of_cropland_kg/ha', 'Value_nutrient_3103_P2O5_Use_per_capita_kg/cap', 'Value_nutrient_3103_P2O5_Use_per_value_of_agricultural_production_g/Int$', 'Value_nutrient_3104_K2O_Agricultural_Use_t', 'Value_nutrient_3104_K2O_Use_per_area_of_cropland_kg/ha', 'Value_nutrient_3104_K2O_Use_per_capita_kg/cap', 'Value_nutrient_3104_K2O_Use_per_value_of_agricultural_production_g/Int$', 'Value_6601_land_area_ha', 'Value_6610_agricultural_land_ha', 'Value_6620_cropland_ha', 'Value_6621_arable_land_ha', 'Value_6650_permanent_crops_ha', 'Value_6655_permanent_meadows_and_pastures_ha', 'Value_pesticide_agricultural_use_t', 'Value_

Ara ens assegurem que no tenim valors nuls en cap registre de les columnes clau ISO3, Country i Year, ja que aquestes són essencials per a l'identificació i agrupació de les dades.


In [4]:
valors_nuls = agri_country_year_df[['ISO3', 'Country', 'Year']].isnull().sum()
print("Valors nuls en les columnes clau:")
print(valors_nuls)
print(f"\nTotal de registres: {len(agri_country_year_df)}")
print(f"Registres amb valors nuls en ISO3, Country o Year: {agri_country_year_df[['ISO3', 'Country', 'Year']].isnull().any(axis=1).sum()}")


Valors nuls en les columnes clau:
ISO3       0
Country    0
Year       0
dtype: int64

Total de registres: 10757
Registres amb valors nuls en ISO3, Country o Year: 0


### Anàlisi de les variables numèriques

### Columnes relatives a la població

#### Comencem per les columnes de població
Les columnes relacionades amb la població són `total_population_No`, `female_population_%`, `male_population_%`, `rural_population_%` i `urban_population_%`. D'aquestes, sabem que `female_population_%` més `male_population_%` sumen 100, i `rural_population_%` més `urban_population_%` sumen aproximadament 100 (o haurien). Això significa que tenim redundància en les dades. Per optimitzar el dataset, podem eliminar algunes d'aquestes columnes segons quines tinguin més dades disponibles. Podem eliminar o bé un dels gèneres i una de les localitzacions, o bé la població total, depenent de quina columna tingui més valors disponibles.


In [5]:
columnes_poblacio = ['total_population_No', 'female_population_%', 'male_population_%', 
                     'rural_population_%', 'urban_population_%']

valors_faltants_poblacio = agri_country_year_df[columnes_poblacio].isnull().sum()
valors_disponibles_poblacio = len(agri_country_year_df) - valors_faltants_poblacio

print("Valors faltants en les columnes de població:")
print(valors_faltants_poblacio)
print("\nValors disponibles en les columnes de població:")
print(valors_disponibles_poblacio)
print("\nPercentatge de valors disponibles:")
print((valors_disponibles_poblacio / len(agri_country_year_df) * 100).round(2))

# % de filas NO nulas donde sumen 100 (redondeado) para les parelles

female = agri_country_year_df['female_population_%'].values
male = agri_country_year_df['male_population_%'].values
mask_genere = ~agri_country_year_df['female_population_%'].isnull() & ~agri_country_year_df['male_population_%'].isnull()
suma_genere = np.round(np.where(mask_genere, female + male, np.nan))
percent100_genere = np.nansum(suma_genere == 100) / np.sum(mask_genere) * 100 if np.sum(mask_genere) > 0 else 0

rural = agri_country_year_df['rural_population_%'].values
urban = agri_country_year_df['urban_population_%'].values
mask_urbs = ~agri_country_year_df['rural_population_%'].isnull() & ~agri_country_year_df['urban_population_%'].isnull()
suma_urbs = np.round(np.where(mask_urbs, rural + urban, np.nan))
percent100_urbs = np.nansum(suma_urbs == 100) / np.sum(mask_urbs) * 100 if np.sum(mask_urbs) > 0 else 0

print(f"\nPercentatge de files NO nules on 'female_population_%' + 'male_population_%' = 100: {percent100_genere:.2f}%")
print(f"Percentatge de files NO nules on 'rural_population_%' + 'urban_population_%' = 100: {percent100_urbs:.2f}%")


Valors faltants en les columnes de població:
total_population_No    0
female_population_%    0
male_population_%      0
rural_population_%     0
urban_population_%     0
dtype: int64

Valors disponibles en les columnes de població:
total_population_No    10757
female_population_%    10757
male_population_%      10757
rural_population_%     10757
urban_population_%     10757
dtype: int64

Percentatge de valors disponibles:
total_population_No    100.0
female_population_%    100.0
male_population_%      100.0
rural_population_%     100.0
urban_population_%     100.0
dtype: float64

Percentatge de files NO nules on 'female_population_%' + 'male_population_%' = 100: 100.00%
Percentatge de files NO nules on 'rural_population_%' + 'urban_population_%' = 100: 21.83%


Hem observat que per a les columnes de gènere, female_population_% i male_population_% sempre sumen 100, per tant podem eliminar male_population_% i mantenir només female_population_%. En canvi, per a les columnes de localització, rural_population_% i urban_population_% només sumen 100 en un 21.83% dels casos, per la qual cosa no són completament redundants i mantenim ambdues columnes.


In [6]:
agri_country_year_df = agri_country_year_df.drop(columns=['male_population_%'])
print("Columna male_population_ eliminada.")


Columna male_population_ eliminada.


Ara convertim les columnes de percentatges a proporcions dividint entre 100. Això transforma els valors de l'interval 0-100 a l'interval 0-1, que és més estàndard per a l'anàlisi i visualització. Apliquem aquesta transformació a female_population_%, rural_population_% i urban_population_%.


In [7]:
columnes_percentatges = ['female_population_%', 'rural_population_%', 'urban_population_%']

for col in columnes_percentatges:
    agri_country_year_df[col] = agri_country_year_df[col] / 100

print("Columnes convertides de percentatges (0-100) a proporcions (0-1):")
for col in columnes_percentatges:
    print(f"  {col}")

print("\nVerificació: valors mínims i màxims després de la conversió:")
for col in columnes_percentatges:
    print(f"  {col}: min={agri_country_year_df[col].min():.4f}, max={agri_country_year_df[col].max():.4f}")


Columnes convertides de percentatges (0-100) a proporcions (0-1):
  female_population_%
  rural_population_%
  urban_population_%

Verificació: valors mínims i màxims després de la conversió:
  female_population_%: min=0.2382, max=0.5561
  rural_population_%: min=0.0000, max=1.3189
  urban_population_%: min=0.0235, max=1.1878


Observem que hi ha valors outliers a rural i urban population en alguns casos, que probablement venen d'agrupacions de païssos fetes on hi havia algun valor faltant a les dades originals i la proporció no es va feer sobre un valor total.

Per tractar aquests casos, sumarem les proporcions totals de rural_population_% i urban_population_% i les estandaritzarem. Aprofitant aquest error a les dades, estandaritzem tots els casos per poder considerar tota la societat o bé rural o bé urbana per facilitar els càlculs i visualitzacions.

In [8]:
# Identificar casos on la suma de rural i urban és major que 1
suma_rural_urban = agri_country_year_df['rural_population_%'] + agri_country_year_df['urban_population_%']
casos_a_normalitzar = round(suma_rural_urban, 8) != 1

print(f"Casos on rural_population_% + urban_population_% != 1: {casos_a_normalitzar.sum()}")
print(f"Percentatge de casos a normalitzar: {casos_a_normalitzar.sum() / len(agri_country_year_df) * 100:.2f}%")

# Normalitzar les proporcions dividint per la seva suma quan la suma > 1
agri_country_year_df['rural_population_%'] = np.where(
    casos_a_normalitzar,
    agri_country_year_df['rural_population_%'] / suma_rural_urban,
    agri_country_year_df['rural_population_%']
)
agri_country_year_df['urban_population_%'] = np.where(
    casos_a_normalitzar,
    agri_country_year_df['urban_population_%'] / suma_rural_urban,
    agri_country_year_df['urban_population_%']
)
print("\nNormalització aplicada.")

# Verificació després de la normalització
suma_rural_urban_despres = agri_country_year_df['rural_population_%'] + agri_country_year_df['urban_population_%']
print(f"\nVerificació després de la normalització:")
print(f"  Valors mínims i màxims de rural_population_%: min={agri_country_year_df['rural_population_%'].min():.4f}, max={agri_country_year_df['rural_population_%'].max():.4f}")
print(f"  Valors mínims i màxims de urban_population_%: min={agri_country_year_df['urban_population_%'].min():.4f}, max={agri_country_year_df['urban_population_%'].max():.4f}")
print(f"  Suma mínima i màxima de rural + urban: min={suma_rural_urban_despres.min():.4f}, max={suma_rural_urban_despres.max():.4f}")
print(f"  Casos on la suma != 1 després de la normalització: {(round(suma_rural_urban_despres, 8) != 1).sum()}")


# Podem eliminar la columna urban_population_% que ja no és necessària.
agri_country_year_df = agri_country_year_df.drop(columns=['urban_population_%'])

Casos on rural_population_% + urban_population_% != 1: 10757
Percentatge de casos a normalitzar: 100.00%

Normalització aplicada.

Verificació després de la normalització:
  Valors mínims i màxims de rural_population_%: min=0.0000, max=0.9760
  Valors mínims i màxims de urban_population_%: min=0.0240, max=1.0000
  Suma mínima i màxima de rural + urban: min=1.0000, max=1.0000
  Casos on la suma != 1 després de la normalització: 0


Amb això, donem per tancat l'anàlisi de les columnes relatives a la població.

#### Columnes sobre empleabilitat

Les columnes relacionades amb l'empleabilitat es poden agrupar en tres blocs. El primer bloc és el valor afegit per treballador, que mesura la productivitat del treballador agrícola. El segon bloc són les dades d'ocupació total en agricultura, amb dues fonts diferents: 21111 que representa crop and animal production i 21144 que representa agriculture segons estimacions modelades de l'ILO. Ambdues fonts tenen dades per a dones, homes i total. El tercer bloc són els percentatges d'ocupació en agricultura, també amb dades per a dones, homes i total.


Consistència interna femenina, masculina i total

En cada grup hem de comprovar si total és aproximadament igual a female més male. Per a les fonts 21111 i 21144, cal verificar aquesta consistència. Si la diferència és propera a zero, està bé. Si és massa gran, hi ha inconsistència en les dades. Si és negativa, pot indicar valors invertits o errors d'importació o exportació.


In [9]:
# Creem una funció per a comprovar la consistència interna d'un indicador d'ocupació donat.
def check_internal_consistency(
    agri_country_year_df: pd.DataFrame, 
    employment_indicator_code: str
    ) -> None:
    """
    Comprova la consistència interna d'un indicador d'ocupació donat.
    """

    # Prefixos llargs per a cada indicador (nom base de la columna)
    prefix_map = {
        "21111": "Value_21111_employment_in_crop_and_animal_production_hunting_and_related_service_activities",
        "21144": "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates"
    }

    if employment_indicator_code not in prefix_map:
        raise ValueError(f"Codi '{employment_indicator_code}' no està definit a prefix_map")

    prefix = prefix_map[employment_indicator_code]

    # Construcció dinamica dels noms complets de columna
    col_female = f"{prefix}_female_No"
    col_male   = f"{prefix}_male_No"
    col_total  = f"{prefix}_total_No"

    # Filtre per files amb valors no nuls
    mask = (~agri_country_year_df[col_female].isnull() &
            ~agri_country_year_df[col_male].isnull() &
            ~agri_country_year_df[col_total].isnull())
    
    null_mask = (agri_country_year_df[col_female].isnull() &
            agri_country_year_df[col_male].isnull() &
            agri_country_year_df[col_total].isnull())

    # Diferència: total - (dones + homes)
    diff_series = (
        agri_country_year_df[col_total]
        - (agri_country_year_df[col_female] + agri_country_year_df[col_male])
    ).where(mask)
    diferencia = diff_series.dropna()

    print(f"\nConsistència interna per a {employment_indicator_code}:")
    total_rows = len(agri_country_year_df)
    print(f"  % registres amb les tres columnes no nules: {mask.sum() / total_rows * 100:.2f}%")
    print(f"  % registres amb les tres columnes nules: {null_mask.sum() / total_rows * 100:.2f}%")
    print(f"  Diferència mitjana: {diferencia.mean():.2f}")
    print(f"  Diferència mediana: {diferencia.median():.2f}")
    print(f"  Diferència màxima: {diferencia.max():.2f}")
    print(f"  Diferència mínima: {diferencia.min():.2f}")
    print(f"  Desviació estàndard: {diferencia.std():.2f}")
    print(f"  % |diferència||||| < 1: {(diferencia.abs() < 1).mean() * 100:.2f}%")
    print(f"  % 1 <= |diferència| < 10: {((diferencia.abs() >= 1) & (diferencia.abs() < 10)).mean() * 100:.2f}%")
    print(f"  Registres amb diferència negativa: {(round(diferencia, 2) < 0).sum()}")


In [10]:
# Creem una funció per a aplicar la consistència interna a un indicador d'ocupació donat.
def enforce_gender_consistency(
        df: pd.DataFrame, 
        indicator_code: str
    ) -> pd.DataFrame:
    """
    Corregeix male/female perquè compleixin:
        male + female = total
    Només en files amb valors no nuls.
    """

    prefix_map = {
        "21111": "Value_21111_employment_in_crop_and_animal_production_hunting_and_related_service_activities",
        "21144": "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates"
    }

    if indicator_code not in prefix_map:
        raise ValueError(f"Codi '{indicator_code}' no està definit a prefix_map")

    prefix = prefix_map[indicator_code]

    col_f = f"{prefix}_female_No"
    col_m = f"{prefix}_male_No"
    col_t = f"{prefix}_total_No"

    # Mascara general: totes tres columnes no nules
    mask = (
        df[col_f].notnull() &
        df[col_m].notnull() &
        df[col_t].notnull()
    )

    # Suma original 
    suma = df[col_f] + df[col_m]

    # Evitar divisions per zero
    mask_valid = mask & (suma != 0)

    # Proporcions
    prop_f = (df[col_f] / suma).where(mask_valid, df[col_f])
    prop_m = (df[col_m] / suma).where(mask_valid, df[col_m])

    # Nous valors
    new_f = (prop_f * df[col_t]).where(mask_valid, df[col_f])
    new_m = (prop_m * df[col_t]).where(mask_valid, df[col_m])

    df[col_f] = new_f
    df[col_m] = new_m

    print(f"Consistència aplicada a {mask_valid.sum()} registres.")
    return df


In [11]:
# Verificar consistència interna per a 21111
check_internal_consistency(agri_country_year_df, "21111")


Consistència interna per a 21111:
  % registres amb les tres columnes no nules: 16.79%
  % registres amb les tres columnes nules: 82.88%
  Diferència mitjana: 76.66
  Diferència mediana: 0.00
  Diferència màxima: 69300.00
  Diferència mínima: -420.00
  Desviació estàndard: 1848.53
  % |diferència||||| < 1: 73.09%
  % 1 <= |diferència| < 10: 0.11%
  Registres amb diferència negativa: 234


Les tres columnes associades a l’indicador 21111 (`employment in crop and animal production, hunting and related service activities` – female, male i total)
presenten problemes greus de qualitat de les dades que comprometen l’anàlisi visual.

Des del punt de vista de qualitat de dades, estadística descriptiva i coherència interna:

1. Percentatge de valors nuls molt elevat: Només el 16.79% dels registres contenen les tres columnes amb valors, mentre que el 82.88% tenen tots els camps nuls. Aquest nivell de manca de dades impedeix:

- Construir sèries temporals fiables.
- Comparar països de manera consistent.
- Generar visualitzacions interpretables sense grans interrupcions.

La discontinuïtat temporal fa que l’indicador no sigui utilitzable en cap anàlisi agrícola a escala internacional.

2. Inconsistència interna entre les variables: Les diferències entre la suma de valors (female + male) i el total mostren anomalies importants:

- Diferència màxima: 69.300
- Diferència mínima: -420
- Desviació estàndard: 1.848,53
- Només un 73.09% de registres presenten una diferència absoluta inferior a 1
- Hi ha 242 casos amb diferència negativa

Aquesta falta de coherència indica problemes greus de report de dades, incompatibilitats entre fonts i errors de mesura. En un indicador demogràfic, la relació total = female + male hauria de complir-se sempre, fet que no succeeix.

3. La imputació seria inviable: Amb més del 80% de valors nuls i incoherències extremes, no és possible aplicar tècniques d’imputació acceptables:

- No hi ha sèries temporals suficients per aplicar forward-fill o interpolació.
- No hi ha estabilitat per país ni per regió.
- Els models predictius generarien valors artificials i sense fonament.

La imputació només introduiria soroll i degradaria la qualitat del dataset.

4. Impacte negatiu en Visualitzacions de Dades: Incloure aquestes variables afegeix soroll aleatori,  dificulta la visualització, pot desestabilitzar mètodes de reducció dimensional i redueix la interpretabilitat dels resultats.

Les variables no aporten informació útil i sí molts problemes potencials. Per tant, les columnes de l’indicador 21111 han de ser eliminades perquè presenten un percentatge de valors nuls molt elevat, una incoherència interna severa i una manca total de continuïtat temporal. Aquestes deficiències les converteixen en variables inadequades per a visualització de dades, ja que només afegeixen distorsions i no aporten informació fiable.


In [12]:
# Eliminar las 3 columnas asociadas al indicador 21111 del dataframe agri_country_year_df

col_female_21111 = "Value_21111_employment_in_crop_and_animal_production_hunting_and_related_service_activities_female_No"
col_male_21111 = "Value_21111_employment_in_crop_and_animal_production_hunting_and_related_service_activities_male_No"
col_total_21111 = "Value_21111_employment_in_crop_and_animal_production_hunting_and_related_service_activities_total_No"

agri_country_year_df = agri_country_year_df.drop(
    [col_female_21111, col_male_21111, col_total_21111],
    axis=1,
    errors="ignore"
)


In [13]:
# Verificar consistència interna per a 21144
check_internal_consistency(agri_country_year_df, "21144")


Consistència interna per a 21144:
  % registres amb les tres columnes no nules: 54.30%
  % registres amb les tres columnes nules: 45.70%
  Diferència mitjana: -0.18
  Diferència mediana: 0.00
  Diferència màxima: 10.00
  Diferència mínima: -10.00
  Desviació estàndard: 5.08
  % |diferència||||| < 1: 74.15%
  % 1 <= |diferència| < 10: 0.09%
  Registres amb diferència negativa: 807


Tot i que l’indicador 21144 (`employment in agriculture – ILO modelled estimates`, per sexe i total) presenta un percentatge moderat de valors nuls, els resultats de la comprovació de consistència interna mostren que és una variable utilitzable i estadísticament coherent dins del context de dades agrícoles globals.

El 54.30% dels registres contenen les tres columnes amb valors. Tot i no ser complet, aquest percentatge permet construir sèries temporals per a una part significativa dels països, fer comparacions regionals, generar visualitzacions consistents sense buits excessius o aplicar tècniques d’imputació raonables.


Les diferències entre total, female i male són petites i controlades:

- diferència mitjana: -0.18
- diferència màxima: 10
- diferència mínima: -10
- el 74.15% dels casos tenen |female + male − total| < 1
- només un 0.09% de casos tenen discrepàncies moderades (entre 1 i 10)

Això indica que les dades segueixen una lògica coherent de desagregació per sexes i que no hi ha errors de magnitud com en el cas de l’indicador 21111.


Per últim, atès que més de la meitat de les observacions són completes i les sèries temporals solen ser estables per país, es poden aplicar tècniques d’imputació com ara:

- forward-fill / backward-fill a nivell de país,
- interpolació temporal,
- models simples basats en tendències històriques.

Aquestes tècniques serien robustes i no introduirien distorsions significatives.


A més, l’indicador 21144 és rellevant per a l’estudi agrícola perquè descriu la força laboral agrària, una variable clau en l’anàlisi socioeconòmica, té correlaciona amb producció, estacionalitat i evolució del sector agrícola. Això permet visualitzacions significatives sobre l’evolució del treball agrícola.

In [14]:
# Apliquem la funció enforce_gender_consistency per tenir consistencia interna en el dataframe agri_country_year_df
agri_country_year_df = enforce_gender_consistency(agri_country_year_df, "21144")

# Comprovem la consistència interna de l'indicador 21144
check_internal_consistency(agri_country_year_df, "21144")

Consistència aplicada a 5841 registres.

Consistència interna per a 21144:
  % registres amb les tres columnes no nules: 54.30%
  % registres amb les tres columnes nules: 45.70%
  Diferència mitjana: -0.00
  Diferència mediana: 0.00
  Diferència màxima: 0.00
  Diferència mínima: -0.00
  Desviació estàndard: 0.00
  % |diferència||||| < 1: 100.00%
  % 1 <= |diferència| < 10: 0.00%
  Registres amb diferència negativa: 0


Veiem ara a quins anys tenim els nuls, per veure si aquest 45.7% de nuls venen a anys concrets o depenen més dels païssos.
Fem una funció per poder-la utilitzar en altres variables.

In [15]:
def null_percentages_by_year(
    df: pd.DataFrame,
    cols: list[str]
) -> dict:
    """
    Calcula el % de nuls per any només per a les columnes indicades en `cols`.
    
    Parameters
    ----
    df : pandas.DataFrame
    cols : list of str
        Llista de columnes sobre les quals calcular el % de nuls.
    
    Returns
    -
    dict :
        - pct_nulls_per_year : Series amb el % de nuls per any
        - llistes classificades per intervals
    """

    # Percentatge de nuls per any (promig dels % de nuls de cada columna)
    year_null_pct = (
        df.groupby("Year")[cols]
          .apply(lambda x: x.isnull().mean().mean() * 100)
          .round(2)
    )

    # Classificació per intervals
    years_0       = year_null_pct[year_null_pct == 0].index.tolist()
    years_0_1     = year_null_pct[(year_null_pct > 0) & (year_null_pct < 1)].index.tolist()
    years_1_10    = year_null_pct[(year_null_pct >= 1) & (year_null_pct < 10)].index.tolist()
    years_10_25   = year_null_pct[(year_null_pct >= 10) & (year_null_pct < 25)].index.tolist()
    years_25_50   = year_null_pct[(year_null_pct >= 25) & (year_null_pct < 50)].index.tolist()
    years_50_75   = year_null_pct[(year_null_pct >= 50) & (year_null_pct < 75)].index.tolist()
    years_75_plus = year_null_pct[year_null_pct >= 75].index.tolist()

    return {
        "pct_nulls_per_year": year_null_pct.sort_index(),
        "years_0_pct": years_0,
        "years_0_1_pct": years_0_1,
        "years_1_10_pct": years_1_10,
        "years_10_25_pct": years_10_25,
        "years_25_50_pct": years_25_50,
        "years_50_75_pct": years_50_75,
        "years_75_plus_pct": years_75_plus
    }


In [16]:
cols_21144 = [
    "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_female_No",
    "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_male_No",
    "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_total_No"
]

result = null_percentages_by_year(agri_country_year_df, cols_21144)

result_without_pct = {k: v for k, v in result.items() if k != "pct_nulls_per_year"}
print(result_without_pct)

{'years_0_pct': [], 'years_0_1_pct': [], 'years_1_10_pct': [1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], 'years_10_25_pct': [2022, 2023], 'years_25_50_pct': [], 'years_50_75_pct': [], 'years_75_plus_pct': [1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990]}


L’anàlisi del percentatge de registres completament nuls per any mostra un patró molt clar en la qualitat de la informació al llarg del temps. Els resultats obtinguts són els següents:

- **years_0_pct**: cap any té el 0% de registres nuls (és a dir, no hi ha cap any completament ple).
- **years_0_1_pct**: cap any presenta menys d’1% de registres nuls.
- **years_1_10_pct**: la majoria dels anys recents (1991–2021) tenen una proporció molt baixa de registres nuls (entre l’1% i el 10%). Això indica una qualitat de dades alta i una cobertura molt estable durant més de tres dècades.
- **years_10_25_pct**: només els anys 2022 i 2023 presenten una lleugera degradació de la cobertura (entre 10% i 25% de nuls), possiblement perquè són anys molt recents i encara no estan completament reportats.
- **years_25_50_pct** i **years_50_75_pct**: no hi ha anys en aquests intervals.
- **years_75_plus_pct**: tots els anys entre 1961 i 1990 tenen més del 75% de registres completament nuls, mostrant que la cobertura històrica és molt deficient i no apta per a anàlisi rigorós.

La informació indica clarament que les dades són sòlides i pràcticament completes a partir de l’any 1991, mentre que els anys anteriors presenten una qualitat insuficient per a un ús fiable en models o visualitzacions. Per aquest motiu:

- Si altres columnes mostren un patró similar, ens quedarem només amb els registres de 1991 en endavant, cosa que ens proporciona més de 30 anys de dades continuades, perfectament adequats per a anàlisi agrícola, visualització i imputació.
- Si altres columnes tenen dades completes des de l’inici, ens plantejarem dues opcions:
  1. Eliminar només aquesta variable i conservar sèries més llargues per la resta.
  2. Imputar els anys anteriors a 1990 i mantenir dues versions del dataset:
     - un dataset complet des de 1961, però amb imputació en aquesta variable;
     - un dataset "clean" 1991–present, sense imputacions i amb qualitat homogènia.

En qualsevol cas, la decisió final es basarà en repetir aquest mateix anàlisi per la resta de variables i assegurar una coherència temporal adequada per a la visualització de dades.


Analitzarem ara les següents variables de percentatge d'ocupació agrícola segons ILO:

 - Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_female_%
 - Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_male_%
 - Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_total_%

En primer lloc, examinarem el percentatge de nuls considerant només aquells registres on les tres columnes siguin alhora no nules (és a dir, creant una mask que exigeixi que les tres tinguin valor).

Posteriorment, veurem la relació amb les variables absolutes anteriors: comprovarem si els registres on aquestes tres variables de percentatge són simultàniament nuls coincideixen amb els registres on les tres variables absolutes corresponents també són nuls. Això ens permetrà entendre la coherència o possibles buits sistemàtics en la cobertura de dades entre les dues fonts.


In [17]:
# Variables amb percentatge d'ocupació agrícola segons ILO (percentatges)
cols_21156 = [
    "Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_female_%",
    "Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_male_%",
    "Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_total_%"
]

# Calculem la mask de files en què totes tres són NO nuls alhora
mask_21156 = agri_country_year_df[cols_21156].notnull().all(axis=1)

# Comptem el percentatge de registres on totes tres tenen valor
pct_valid_total_21156 = mask_21156.mean()

print("Percentatge total de registres on les tres variables % ILO són no nules:")
print(f"{pct_valid_total_21156 * 100:.1f}%")

null_percentages_by_year_share = null_percentages_by_year(agri_country_year_df, cols_21156)
result_without_pct = {k: v for k, v in result.items() if k != "pct_nulls_per_year"}
print(result_without_pct)

# Ara, per veure la coherència amb les variables absolutes anteriors:
mask_21144 = agri_country_year_df[cols_21144].notnull().all(axis=1)

# Files on les tres percentatges SÓN simultàniament nuls
mask_21156_allnull = agri_country_year_df[cols_21156].isnull().all(axis=1)
# Files on les tres absolutes SÓN simultàniament nuls
mask_21144_allnull = agri_country_year_df[cols_21144].isnull().all(axis=1)

# Quina proporció de files amb tots 3 % nuls té també tots 3 absoluts nuls?
if mask_21156_allnull.sum() > 0:
    coincidencia_nuls = (mask_21156_allnull & mask_21144_allnull).sum() / mask_21156_allnull.sum()
else:
    coincidencia_nuls = float('nan')
print(f"\nProporció de files on les 3 variables de % ILO són nules i també totes tres absolutes són nules: {coincidencia_nuls:.2%}")

# I a la inversa: quina proporció de files amb tots 3 absoluts nuls té també tots 3 % nuls?
if mask_21144_allnull.sum() > 0:
    coincidencia_nuls_inv = (mask_21144_allnull & mask_21156_allnull).sum() / mask_21144_allnull.sum()
else:
    coincidencia_nuls_inv = float('nan')
print(f"Proporció de files on les 3 variables absolutes són nules i també totes tres % són nules: {coincidencia_nuls_inv:.2%}")

Percentatge total de registres on les tres variables % ILO són no nules:
54.3%
{'years_0_pct': [], 'years_0_1_pct': [], 'years_1_10_pct': [1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], 'years_10_25_pct': [2022, 2023], 'years_25_50_pct': [], 'years_50_75_pct': [], 'years_75_plus_pct': [1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990]}

Proporció de files on les 3 variables de % ILO són nules i també totes tres absolutes són nules: 100.00%
Proporció de files on les 3 variables absolutes són nules i també totes tres % són nules: 100.00%


Observem que són exactament els mateixos registres, i com a més, tenim la columna `total_population_No`, aquestes 6 columnes ens aporten informació irrellevant.

Per tant, com les fonts són diferents, el qué farem será la següent estratègia:
- Aprofitar que hi ha un 0% de valores nuls a `total_population_No` y `female_population_No`.
- Utilitzar els valors de `21144` amb els valors totals de població dedicada a l'agricultura.
- Calcular el `share_of_employmet_in_agriculture` pel gènere femeni i pel total com: `21144_indicator / total`.
    - Calcular el masculí seria introduir informació irrellevant, ja que pot ser calculat com: `male_share_of_employmet_in_agriculture =  [total_share_of_employmet_in_agriculture - female_population_% * female_share_of_employmet_in_agriculture] / (1 - female_population_%)`
- Eliminar les variables per `21144` i `21156`


Per tant:
- female_agri_share = female_employment_in_agriculture / total_female_population
- total_agri_share = total_employment_in_agriculture / total_population_No

In [18]:
# 1. Calcular les noves variables
agri_country_year_df["female_share_of_employment_in_agriculture"] = (
    agri_country_year_df["Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_female_No"]
    / (agri_country_year_df["total_population_No"] * agri_country_year_df["female_population_%"])
)

agri_country_year_df["total_share_of_employment_in_agriculture"] = (
    agri_country_year_df["Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_total_No"]
    / agri_country_year_df["total_population_No"]
)

# 2. Eliminar columnes originals
cols_to_drop = [
    "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_female_No",
    "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_male_No",
    "Value_21144_employment_in_agriculture_-_ilo_modelled_estimates_total_No",
    "Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_female_%",
    "Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_male_%",
    "Value_21156_share_of_employment_in_agriculture_in_total_employment_-_ilo_modelled_estimates_total_%"
]

agri_country_year_df = agri_country_year_df.drop(columns=[c for c in cols_to_drop if c in agri_country_year_df.columns])


## Variables econòmiques relacionades amb el sector agrícola

Disposem de tres variables econòmiques proporcionades per FAOSTAT/World Bank que descriuen aspectes macroeconòmics del sector agrícola, forestal i pesquer. Com que el seu significat exacte no és crítica per a l’objectiu actual i poden ser interpretades com a indicadors econòmics generals, els assignem una descripció genèrica:

- **economic_indicator_agri_value_added_per_worker_usd**  
- **economic_indicator_agri_share_gdp_percent**  
- **economic_indicator_agri_annual_growth_percent**

Calculem el percentatge de valors nuls de cadascuna d'aquestes variables per entendre la seva qualitat i disponibilitat de dades. Aquesta informació serà útil a l’hora de decidir si mantenir-les, imputar-les o eliminar-les en fases posteriors de modelatge.


In [19]:
df = agri_country_year_df.copy()

# Renombrar variables amb noms genèrics
rename_map = {
    "Value_21085_agriculture_value_added_per_worker_constant_2015_ususd_total_USD":
        "agriculture_value_added_per_worker_usd",

    "agriculture_forestry_fishing_value_added_%_GDP":
        "agriculture_share_gdp_percent",

    "agriculture_forestry_fishing_value_added_%_annual_growth":
        "agriculture_annual_growth_percent"
}

df = df.rename(columns=rename_map)

# Variables a avaluar 
econ_vars = list(rename_map.values())

print("Percentatge de valors nuls per variable econòmica:\n")
print((df[econ_vars].isna().mean() * 100).round(2))

agri_country_year_df = df

Percentatge de valors nuls per variable econòmica:

agriculture_value_added_per_worker_usd    47.64
agriculture_share_gdp_percent             23.57
agriculture_annual_growth_percent         28.73
dtype: float64


Un cop identificades les tres variables econòmiques agregades del sector agrícola, forestal i pesquer, el primer pas consisteix a avaluar la seva completitud, ja que les 3 tenen valors faltants. Atès que aquestes variables són sèries temporals per país, el mètode d’imputació més coherent és la interpolació lineal al llarg del temps dins de cada país.


In [20]:
df = agri_country_year_df.copy()

df = df.sort_values(["Country", "Year"])

for col in econ_vars:

    # interpolació temporal segura amb transform
    df[col] = (
        df.groupby("Country")[col]
          .transform(lambda s: s.interpolate(method="linear"))
    )

    # Límit mínim i màxim segons la naturalesa de la variable
    if col == "economic_indicator_agri_value_added_per_worker_usd":
        # variable en nivells econòmics : mínim = 0
        df[col] = df[col].clip(lower=0)

    else:
        # variables en percentatges, rang 0-100
        df[col] = df[col].clip(lower=0, upper=100)


# 4. % de nuls després
print("Percentatge de NaNs després de la imputació:")
print((df[econ_vars].isna().mean() * 100).round(2), "\n")

agri_country_year_df = df

Percentatge de NaNs després de la imputació:
agriculture_value_added_per_worker_usd    46.99
agriculture_share_gdp_percent             22.39
agriculture_annual_growth_percent         28.02
dtype: float64 



D’aquestes tres variables, agriculture_value_added_per_worker_usd és la que presenta més problemes, ja que té el percentatge més alt de valors nuls (47%), gairebé la meitat de totes les observacions. A més, no és un percentatge, sinó un valor absolut econòmic que pot variar enormement entre països, cosa que dificulta qualsevol imputació robusta. Encara que tenim variables de població i d’ús del sòl, que permeten construir mètriques més coherents que aquesta, sense una referència comuna, imputar-la podria introduir biaixos greus, especialment en països amb economies agrícoles petites o poc reportades.

Per aquests motius, decidim eliminar aquesta variable, considerant que el risc d’incorporar soroll o informació distorsionada és més alt que el benefici potencial.

In [21]:
df = agri_country_year_df.copy()

# Variable a eliminar
col_to_drop = "agriculture_value_added_per_worker_usd"

# Eliminar del dataframe
df = df.drop(columns=[col_to_drop], errors="ignore")

agri_country_year_df = df

## Tractament de les Variables d’Emissions Agrícoles en CO₂ Equivalent (FAO AR5 Tier 1)

In [22]:
agri_country_year_df[[
    "Value_1709_agricultural_soils_CO2eq_AR5_kt_FAO_TIER_1",
    "Value_5058_enteric_fermentation_CO2eq_AR5_kt_FAO_TIER_1",
    "Value_5059_manure_management_CO2eq_AR5_kt_FAO_TIER_1",
    "Value_5060_rice_cultivation_CO2eq_AR5_kt_FAO_TIER_1",
    "Value_5061_synthetic_fertilizers_CO2eq_AR5_kt_FAO_TIER_1",
    "Value_5064_crop_residues_CO2eq_AR5_kt_FAO_TIER_1",
    "Value_5085_emissions_from_livestock_CO2eq_AR5_kt_FAO_TIER_1"
]].head()

,Value_1709_agricultural_soils_CO2eq_AR5_kt_FAO_TIER_1,Value_5058_enteric_fermentation_CO2eq_AR5_kt_FAO_TIER_1,Value_5059_manure_management_CO2eq_AR5_kt_FAO_TIER_1,Value_5060_rice_cultivation_CO2eq_AR5_kt_FAO_TIER_1,Value_5061_synthetic_fertilizers_CO2eq_AR5_kt_FAO_TIER_1,Value_5064_crop_residues_CO2eq_AR5_kt_FAO_TIER_1,Value_5085_emissions_from_livestock_CO2eq_AR5_kt_FAO_TIER_1
32,2648.1450,6739.1268,431.2264,823.2,5.512,284.4510,9528.5352
33,2619.1275,6868.6968,442.0031,823.2,5.512,286.6240,9637.6942
34,2663.8860,7163.1980,462.2652,823.2,5.512,267.3585,10016.4787
35,2719.9070,7253.8200,470.5985,862.4,5.512,288.3200,10150.4670
36,2784.3550,7436.7440,487.0982,862.4,5.512,291.4470,10411.2382


### Càlcul i simplificació de les emissions agrícoles en CO₂ equivalent (AR5)

Per tal de treballar amb les dades d’emissions agrícoles de manera clara i coherent, he revisat les diferents variables relacionades amb les emissions de gasos d’efecte hivernacle en unitats *CO2 equivalent (CO2eq)* segons l’informe **AR5 de l’IPCC**, expressades en **quilotones (kt)**.
https://www.ipcc.ch/assessment-report/ar5/


### Què significa CO2eq_AR5_kt?
- **CO₂eq** indica que diversos gasos (com el metà o el N2O) s’han convertit a una escala comuna de CO2 utilitzant el seu potencial d’escalfament global.  
- **AR5** fa referència al cinquè informe de l’IPCC, que defineix els factors d’equivalència.  

https://es.wikipedia.org/wiki/CO%E2%82%82_equivalente#:~:text=El%20equivalente%20de%20CO2%20o%20equivalente%20de,la%20emisi%C3%B3n%20de%20gases%20de%20efecto%20invernadero.

Això permet comparar emissions procedents de fonts diferents (bestiar, sòls, arròs, etc.) dins d’una mateixa unitat comunament utilitzada en informes climàtics.


#### Estructura de les variables i per què no puc sumar-les totes directament
Les variables relacionades amb emissions agrícoles eren:

- `agricultural_soils_CO2eq`
- `enteric_fermentation_CO2eq`
- `manure_management_CO2eq`
- `rice_cultivation_CO2eq`
- `synthetic_fertilizers_CO2eq`
- `crop_residues_CO2eq`
- `emissions_from_livestock_CO2eq`

L’última (`emissions_from_livestock_CO2eq`) ja és la suma de fermentació entèrica + gestió de fems.

Per tant, no té sentit sumar‐la juntament amb les seves components, perquè generaria doble comptatge.


#### Variables finals creades
##### 1. `emissions_livestock_total_CO2eq_AR5_kt`: Agrupa les emissions del bestiar utilitzant la variable agregada de FAOSTAT (`emissions_from_livestock_CO2eq_AR5_kt`), que ja integra fermentació entèrica i gestió de fems.

##### 2. `emissions_total_agriculture_CO2eq_AR5_kt`
Emissions agrícoles totals, sumant totes les fonts agricules independents:
- sols agrícoles  
- residus de cultiu  
- fertilitzants sintètics  
- cultiu de l’arròs  


#### Eliminació de columnes redundants
Un cop creades les dues variables finals, eliminem: `enteric_fermentation_CO2eq_AR5_kt` i `manure_management_CO2eq_AR5_kt`, ja no són necessàries perquè la variable agregada del bestiar (`5085`) les incorpora de manera fiable segons FAOSTAT.


In [23]:
df = agri_country_year_df.copy()

# 1. Definició de columnes originals
col_soils = 'Value_1709_agricultural_soils_CO2eq_AR5_kt_FAO_TIER_1'
col_enteric = 'Value_5058_enteric_fermentation_CO2eq_AR5_kt_FAO_TIER_1'
col_manure = 'Value_5059_manure_management_CO2eq_AR5_kt_FAO_TIER_1'
col_rice = 'Value_5060_rice_cultivation_CO2eq_AR5_kt_FAO_TIER_1'
col_fert = 'Value_5061_synthetic_fertilizers_CO2eq_AR5_kt_FAO_TIER_1'
col_residues = 'Value_5064_crop_residues_CO2eq_AR5_kt_FAO_TIER_1'
col_livestock = 'Value_5085_emissions_from_livestock_CO2eq_AR5_kt_FAO_TIER_1'

# 2. Crear emissions totals del bestiar
df['emissions_total_livestock_CO2eq_AR5_kt'] = df[col_livestock].fillna(0)

# 3. Crear emissions totals d'agricultura (sense doble comptatge)
df['emissions_total_agriculture_CO2eq_AR5_kt'] = (
    df[col_soils].fillna(0)
    + df[col_rice].fillna(0)
    + df[col_fert].fillna(0)
    + df[col_residues].fillna(0)
)

# 4. Eliminar columnes redundants
cols_to_drop = [col_enteric, col_manure, col_livestock]
df.drop(columns=cols_to_drop, inplace=True)

# 5. Percentatge de valors faltants (per columna)
emissions_cols = [
    'emissions_total_livestock_CO2eq_AR5_kt',
    'emissions_total_agriculture_CO2eq_AR5_kt',
    col_soils,
    col_rice,
    col_fert,
    col_residues
]

missing_percent = df[emissions_cols].isna().mean() * 100

print("% de valors faltants per columna d'emissions resultants")
print(missing_percent.round(2))

# Resultat final
agri_country_year_df = df


% de valors faltants per columna d'emissions resultants
emissions_total_livestock_CO2eq_AR5_kt                       0.00
emissions_total_agriculture_CO2eq_AR5_kt                     0.00
Value_1709_agricultural_soils_CO2eq_AR5_kt_FAO_TIER_1        0.00
Value_5060_rice_cultivation_CO2eq_AR5_kt_FAO_TIER_1         34.36
Value_5061_synthetic_fertilizers_CO2eq_AR5_kt_FAO_TIER_1     0.01
Value_5064_crop_residues_CO2eq_AR5_kt_FAO_TIER_1             5.44
dtype: float64


### Imputació avançada dels valors faltants en les variables d’emissions agrícoles

Per garantir la coherència temporal i evitar biaixos en les sèries anuals d’emissions agrícoles, he aplicat un procés d’imputació basat en interpolació temporal i, en casos extrems, mediana per any.

Les variables tractades són:

- `emissions_rice_cultivation_agricultural_CO2eq_AR5_kt`
- `emissions_synthetic_fertilizers_CO2eq_AR5_kt`
- `emissions_crop_residues_CO2eq_AR5_kt`

1. `rice_cultivation`: interpolació temporal i fallback a mediana per any
Aquesta variable presenta un percentatge elevat de NaNs.  
La lògica aplicada és:

1. Interpolació lineal per país segons la sèrie temporal.  
   Això permet omplir anys intermedis respectant la tendència.

2. Si un país no té cap dada reportada, llavors no podem interpolar.  
   En aquest cas s’imputa la mediana global per any, assegurant coherència entre països.

3. Si la mediana per any també fos NaN (casos rars), s’usa la mediana global total.

2. `synthetic_fertilizers`: interpolació temporal i fallback a mediana per any
Tot i que només un 0,01% són NaN, aplico el mateix mètode per coherència:

- Interpolació lineal per país.  
- Si el país no té dades suficients : mediana global per any.

3. `crop_residues`: interpolació temporal i fallback a mediana per any
Els residus de cultius varien anualment de manera suau. Per tant:

- Interpolació lineal quan hi ha dades parcials.
- Si no hi ha cap dada per a un país : mediana global per any.


In [24]:
df = agri_country_year_df.copy()

# Funció d’imputació amb interpolació + fallback 
def interpolate_with_year_median(series, df_year_indexed, col):
    """
    Interpolació per país si hi ha dades.
    Si no en té cap, mediana per any.
    Si tampoc, mediana global.
    """
    if series.notna().sum() > 0:
        # Interpolació temporal
        result = series.interpolate(method="linear")
        # Fallback per si encara queden NaNs després d'interpolar (principi/final serie)
        if result.isna().any():
            year_idx = result.index.get_level_values("Year")
            median_per_year = df_year_indexed.groupby("Year")[col].median()
            filled = result.fillna(pd.Series(median_per_year.reindex(year_idx).values, index=series.index))
            # Si queda algun NaN, omple amb la mediana global
            final = filled.fillna(df_year_indexed[col].median())
            return final
        else:
            return result
    else:
        # País sense dades, mediana per any
        year_idx = series.index.get_level_values("Year")
        median_per_year = df_year_indexed.groupby("Year")[col].median()
        new_values = pd.Series(median_per_year.reindex(year_idx).values, index=series.index)
        # si encara hi ha NaN, mediana global
        return new_values.fillna(df_year_indexed[col].median())

# Renombrar columnas antes de Imputem
df = df.rename(columns={
    'Value_1709_agricultural_soils_CO2eq_AR5_kt_FAO_TIER_1': 'emissions_soils_CO2eq_AR5_kt',
    'Value_5060_rice_cultivation_CO2eq_AR5_kt_FAO_TIER_1': 'emissions_rice_cultivation_agricultural_CO2eq_AR5_kt',
    'Value_5061_synthetic_fertilizers_CO2eq_AR5_kt_FAO_TIER_1': 'emissions_synthetic_fertilizers_CO2eq_AR5_kt',
    'Value_5064_crop_residues_CO2eq_AR5_kt_FAO_TIER_1': 'emissions_crop_residues_CO2eq_AR5_kt'
})

col_rice = 'emissions_rice_cultivation_agricultural_CO2eq_AR5_kt'
col_fert = 'emissions_synthetic_fertilizers_CO2eq_AR5_kt'
col_residues = 'emissions_crop_residues_CO2eq_AR5_kt'

cols = [col_rice, col_fert, col_residues]

df = df.sort_values(["Country", "Year"])

# Eliminar files amb duplicitat de (Country, Year)
if df.duplicated(subset=["Country", "Year"]).any():
    print("Hi ha files duplicades per (Country, Year). S'eliminen duplicats mantenint la fila amb menys valors Nuls.")
    # Afegeix una columna amb el recompte de NaNs per fila
    df['_num_nulls'] = df.isnull().sum(axis=1)
    # Ordena perquè la fila amb menys NaNs vingui primer, després elimina duplicats mantenint la millor
    df = (
        df.sort_values(by=["Country", "Year", "_num_nulls"])
          .drop_duplicates(subset=["Country", "Year"], keep="first")
          .drop(columns="_num_nulls")
          .copy()
    )
else:
    # Encara que no hi hagi duplicats, assegurem que no hi hagi la columna auxiliar
    df = df.copy()

# Imputació per cada columna utilitzant DataFrame
df = df.set_index(["Country", "Year"])
for col in cols:
    df[col] = (
        df.groupby(level="Country")[col]
          .transform(lambda s: interpolate_with_year_median(s, df, col))
    )
    # Assegurem que no quedi cap NaN (encara un cop tot l'anterior)
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

df = df.reset_index()

missing_after = df[cols].isna().mean() * 100
print("Percentatge de NaNs després de la imputació:")
print(missing_after.round(2))
assert missing_after.max() == 0, "Encara queden NaNs després de la imputació!"

agri_country_year_df = df

Hi ha files duplicades per (Country, Year). S'eliminen duplicats mantenint la fila amb menys valors Nuls.
Percentatge de NaNs després de la imputació:
emissions_rice_cultivation_agricultural_CO2eq_AR5_kt    0.0
emissions_synthetic_fertilizers_CO2eq_AR5_kt            0.0
emissions_crop_residues_CO2eq_AR5_kt                    0.0
dtype: float64


## Processament de les variables de `Land Use`

En aquest pas reorganitzo les variables d’ús del sòl per simplificar l’estructura i crear una jerarquia més clara. Les dades originals eren:

- `land_area_ha` — superfície total del país  
- `agricultural_land_ha` — terres agrícoles  
- `cropland_ha` — terres de cultiu  
- `arable_land_ha` — terra cultivable anual  
- `permanent_crops_ha` — cultius permanents  
- `permanent_meadows_and_pastures_ha` — prats i pastures permanents  

#### 1. Renombrament de variables
Per facilitar la lectura i anàlisi, les variables s’han renombrat a noms més curts i coherents.

#### 2. Simplificació de jerarquies
Originalment, la jerarquia tenia quatre nivells dins de les terres de cultiu:


level_1__|||level_2__________|||level_3________________________|||level_4

land_area|||agricultural_land|||cropland_______________________|||arable_land

land_area|||agricultural_land|||cropland_______________________|||permanent_crops

land_area|||agricultural_land|||permanent_meadows_and_pastures



A efectes pràctics, arable land + permanent crops = cropland, de manera que fusiono els dos components de *cropland* en una sola variable: cropland_simplified = arable_land + permanent_crops


En aquesta suma, els valors faltants es tracten com a 0 per mantenir la coherència i evitar pèrdua d’informació quan un país només reporta una de les dues.

Després de la fusió, la jerarquia queda simplificada a tres nivells: cropland_simplified = arable_land + permanent_crops

level_1__|||level_2__________|||level_3

land_area|||agricultural_land|||cropland

land_area|||agricultural_land|||permanent_meadows_and_pastures

In [25]:
df = agri_country_year_df.copy()

# 1. Renombrar variables
rename_map = {
    'Value_6601_land_area_ha': 'land_area_ha',
    'Value_6610_agricultural_land_ha': 'agricultural_land_ha',
    'Value_6620_cropland_ha': 'cropland_ha',
    'Value_6621_arable_land_ha': 'arable_land_ha',
    'Value_6650_permanent_crops_ha': 'permanent_crops_ha',
    'Value_6655_permanent_meadows_and_pastures_ha': 'permanent_meadows_and_pastures_ha'
}

df = df.rename(columns=rename_map)

# 2. Crear nou cropland 
df['cropland_ha'] = df['cropland_ha'].combine_first(
    df['arable_land_ha'].fillna(0) + df['permanent_crops_ha'].fillna(0)
)

df.drop(columns=['arable_land_ha', 'permanent_crops_ha'], inplace=True)

# 3. Percentatge de nuls en les variables finals
cols_to_check = [
    'land_area_ha',
    'agricultural_land_ha',
    'cropland_ha',
    'permanent_meadows_and_pastures_ha'
]

missing_percent = df[cols_to_check].isna().mean() * 100

print("Percentatge de valors faltants en les variables de land use:")
print(missing_percent.round(2))

# Actualitzar dataset
agri_country_year_df = df

Percentatge de valors faltants en les variables de land use:
land_area_ha                         0.00
agricultural_land_ha                 0.00
cropland_ha                          0.00
permanent_meadows_and_pastures_ha    3.41
dtype: float64


### Imputació dels valors faltants en `permanent_meadows_and_pastures_ha`


El 3,37% de valors faltants en `permanent_meadows_and_pastures_ha` és relativament reduït, però suficient per afectar les anàlisis si no es tracta adequadament.  En lloc d’imputar amb la mitjana o mediana del país, que generaria valors artificials i trencaria la dinàmica temporal, utilitzem un mètode més coherent per a sèries anuals: Interpolació lineal temporal per país.

Per a cada país: Busquem el valor anterior conegut, busquem el valor posterior conegut, imputem els anys intermedis mitjançant una interpolació lineal.


Això garanteix que l’evolució de les pastures siga suau i coherent, no creem passos abruptes ni valors arbitràriament constants, i respectem la lògica del canvi d’ús del sòl al llarg del temps.

Aquest mètode és especialment adequat per variables com les pastures, que no canvien bruscament d’un any a un altre i on la degradació o expansió del territori sol ser progressiva.


In [26]:
df = agri_country_year_df.copy()

col_pastures = "permanent_meadows_and_pastures_ha"

# Imputació per interpolació lineal per país
df = df.sort_values(["Country", "Year"])

df[col_pastures] = (
    df.groupby("Country")[col_pastures]
      .apply(lambda x: x.interpolate(method="linear"))
      .reset_index(level=0, drop=True)
)

# Si encara quedara algun NaN (p. ex. país sense cap valor), imputar mediana global
if df[col_pastures].isna().sum() > 0:
    df[col_pastures] = df[col_pastures].fillna(df[col_pastures].median())

# Recalcul del % de NaNs per verificar
missing_after = df[col_pastures].isna().mean() * 100
print(f"% de NaNs en pastures després de la imputació: {missing_after:.2f}%")

# Actualitzar dataset final
agri_country_year_df = df

% de NaNs en pastures després de la imputació: 0.00%


## Tractament de les variables percentuals d’ús del sòl del data set del clima

Després de processar i netejar les variables principals de *Land Use*, ja disposem de quatre variables bàsiques completament lliures de valors nuls:

- `land_area_ha`
- `agricultural_land_ha`
- `cropland_ha`
- `permanent_meadows_and_pastures_ha`

Aquestes variables permeten descriure tot l’estructura d’ús del sòl d’un país sense inconsistències.  
A continuació analitzem el segon conjunt de variables relacionades amb percentatges i derivacions:

- `agricultural_irrigated_land_%_of_total_agri_land`
- `agricultural_land_%_total_land`
- `arable_land_%_total_land`
- `arable_land_ha_per_person`

Els seus percentatges de valors nuls són:

In [27]:
land_perc_cols = ['agricultural_irrigated_land_%_of_total_agri_land', 'agricultural_land_%_total_land', 'arable_land_%_total_land', 'arable_land_ha_per_person']
agri_country_year_df[land_perc_cols].isna().mean() * 100


agricultural_irrigated_land_%_of_total_agri_land    88.618192
agricultural_land_%_total_land                       4.411626
arable_land_%_total_land                             4.938388
arable_land_ha_per_person                            4.938388
dtype: float64

Tres d’aquestes variables no aporten informació nova, ja que es poden calcular directament a partir de les variables de *Land Use* i altres variables que ja tenim sense nuls:

- `agricultural_land_%_total_land`= `agricultural_land_ha / land_area_ha * 100`

- `arable_land_%_total_land` =  `arable_land_ha / land_area_ha * 100` i es pot derivar a partir de `cropland_ha` i la informació de cultius permanents.

- `arable_land_ha_per_person` = `arable_land_ha / total_population_No`

Aquestes tres variables són redundants i, per coherència, és millor eliminar-les i generar-les sempre que calgui a partir de dades fonamentals i consistents.

Sobre la variable de regadiu, `agricultural_irrigated_land_%_of_total_agri_land` és l’única que no es pot derivar directament de les variables de superfície, ja que requereix informació específica sobre infraestructura d’irrigació. Tanmateix, presenta 88,54% de valors nuls, la qual cosa la converteix en una variable poc fiable, difícil d’imputar i poc útil per a models o anàlisis estadístics.

Per tant, les quatre variables del conjunt són eliminades.


In [28]:
df = agri_country_year_df.copy()

cols_to_drop = [
    "agricultural_irrigated_land_%_of_total_agri_land",
    "agricultural_land_%_total_land",
    "arable_land_%_total_land",
    "arable_land_ha_per_person"
]

df = df.drop(columns=cols_to_drop, errors="ignore")

agri_country_year_df = df

## Simplificació i unificació de les variables de consum de Fertilitzants

Les dades originals de fertilitzants provenen de FAOSTAT i inclouen tres nutrients principals:

- Nitrogen (N)
- Fosfat (P2O5)
- Potassa (K2O)

Per a cadascun d'ells es reporten quatre tipus de mètriques:

- Ús total en tones (Agricultural Use)
- Ús per superfície de cultiu (kg/ha)
- Ús per càpita (kg/cap)
- Ús per unitat de valor agrícola (g/Int$)

1. Reducció de dimensionalitat: agregació de nutrients. Com que el nostre objectiu no és analitzar els nutrients per separat, sinó obtenir una mesura global del consum de fertilitzants, agreguem els tres nutrients en mètriques comunes.  A més, totes les mètriques ja es troben en *unitats comparables* (kg/ha, kg/cap, g/Int$), per tant és coherent sumar-les.

Així, definim:

- `fertilizers_per_area_of_cropland_kg_ha`
- `fertilizers_per_capita_kg_cap`
- `fertilizers_per_value_of_agri_prod_g_Intdollar`


2. Reducció de mètriques: descartem l'ús total en tones. No és necessari mantenir l'ús total (`Agricultural_Use_t`), ja que:
- disposem de la població total, podem derivar `kg/cap`  
- disposem de la superfície agrícola, podem derivar `kg/ha`  
- és redundant i menys interpretable que les mètriques normalitzades.

### 3. Estratègia COALESCE per minimitzar valors NA
Per evitar perdre informació, utilitzem una estratègia de **COALESCE**, és a dir:

- Si existeix la mètrica normalitzada, la utilitzem.
- Si no existeix, la derive’m a partir de l'ús total.

3.1 Per superfície (kg/ha): `fertilizers_per_area =
COALESCE(
suma(N, P2O5, K2O)_per_area_kg/ha,
suma(N, P2O5, K2O)_total_t / cropland_ha * 1000
)`

3.2 Per càpita (kg/cap): `fertilizers_per_capita =
COALESCE(
suma(N, P2O5, K2O)_per_capita_kg/cap,
suma(N, P2O5, K2O)_total_t / (1000 * total_population)
)`

3.3 Per unitat de valor agrícola (g/Int$): En aquest cas, no sempre podem derivar valors consistents, així que només sumem els nutrients quan existeixen.


In [29]:
df = agri_country_year_df.copy()

# 1. Noms de les columnes
nutrients = ["3102_N", "3103_P2O5", "3104_K2O"]

total_cols = [f"Value_nutrient_{n}_Agricultural_Use_t" for n in nutrients]
per_area_cols = [f"Value_nutrient_{n}_Use_per_area_of_cropland_kg/ha" for n in nutrients]
per_cap_cols = [f"Value_nutrient_{n}_Use_per_capita_kg/cap" for n in nutrients]
per_value_cols = [f"Value_nutrient_{n}_Use_per_value_of_agricultural_production_g/Int$" for n in nutrients]

# 2. Sumar nutrients (N + P2O5 + K2O)
df["fert_total_t"] = df[total_cols].sum(axis=1, skipna=True)
df["fert_per_area_kg_ha_raw"] = df[per_area_cols].sum(axis=1, skipna=True)
df["fert_per_capita_kg_cap_raw"] = df[per_cap_cols].sum(axis=1, skipna=True)
df["fert_per_value_g_int_raw"] = df[per_value_cols].sum(axis=1, skipna=True)

# 3. COALESCE per superfície (kg/ha)
df["fertilizers_per_area_of_cropland_kg_ha"] = df["fert_per_area_kg_ha_raw"]

mask_area_missing = df["fertilizers_per_area_of_cropland_kg_ha"].isna()
df.loc[mask_area_missing, "fertilizers_per_area_of_cropland_kg_ha"] = (
    df.loc[mask_area_missing, "fert_total_t"] * 1000 / df.loc[mask_area_missing, "cropland_ha"]
)

# 4. COALESCE per càpita (kg/cap)

df["fertilizers_per_capita_kg_cap"] = df["fert_per_capita_kg_cap_raw"]

mask_cap_missing = df["fertilizers_per_capita_kg_cap"].isna()
df.loc[mask_cap_missing, "fertilizers_per_capita_kg_cap"] = (
    df.loc[mask_cap_missing, "fert_total_t"] * 1000 / df.loc[mask_cap_missing, "total_population_No"]
)

# 5. Per valor agrícola (g/Int$): No podem derivar-ho, només sumem els nutrients disponibles

df["fertilizers_per_value_of_agri_prod_g_Intdollar"] = df["fert_per_value_g_int_raw"]

# 6. Eliminar les columnes intermitges
df.drop(columns=[
    "fert_total_t",
    "fert_per_area_kg_ha_raw",
    "fert_per_capita_kg_cap_raw",
    "fert_per_value_g_int_raw"
], inplace=True)

# Resultat final
agri_country_year_df = df

# Percentatge de nuls de cadascuna
fert_vars = [
    "fertilizers_per_area_of_cropland_kg_ha",
    "fertilizers_per_capita_kg_cap",
    "fertilizers_per_value_of_agri_prod_g_Intdollar"
]
print("\nPercentatge de valors nuls:")
null_pct = agri_country_year_df[fert_vars].isna().mean() * 100
print(null_pct.round(2))


Percentatge de valors nuls:
fertilizers_per_area_of_cropland_kg_ha            0.0
fertilizers_per_capita_kg_cap                     0.0
fertilizers_per_value_of_agri_prod_g_Intdollar    0.0
dtype: float64


In [30]:
# Llista de totes les columnes originals de fertilitzants que vols eliminar
fert_cols_to_drop =  total_cols + per_area_cols + per_cap_cols + per_value_cols

# Eliminar-les del dataframe
agri_country_year_df = agri_country_year_df.drop(columns=fert_cols_to_drop, errors="ignore")

print("Columnes de fertilitzants originals eliminades.")

display(agri_country_year_df.head())

Columnes de fertilitzants originals eliminades.


,Country,Year,ISO3,land_area_ha,agricultural_land_ha,cropland_ha,permanent_meadows_and_pastures_ha,Value_pesticide_agricultural_use_t,Value_6971_herbaceous_crops_ha,Value_6974_tree-covered_areas_ha,...,agriculture_annual_growth_percent,prec_mm_per_year,mean_annual_tas_deg_celsius,female_share_of_employment_in_agriculture,total_share_of_employment_in_agriculture,emissions_total_livestock_CO2eq_AR5_kt,emissions_total_agriculture_CO2eq_AR5_kt,fertilizers_per_area_of_cropland_kg_ha,fertilizers_per_capita_kg_cap,fertilizers_per_value_of_agri_prod_g_Intdollar
0,Afghanistan,1961,AFG,65223000.0,37750000.0,7750000.0,30000000.0,NaN,NaN,NaN,...,NaN,345.95,12.65,NaN,NaN,9528.5352,3761.3080,0.14,0.12,0.43
1,Afghanistan,1962,AFG,65223000.0,37800000.0,7800000.0,30000000.0,NaN,NaN,NaN,...,NaN,277.44,12.33,NaN,NaN,9637.6942,3734.4635,0.14,0.12,0.42
2,Afghanistan,1963,AFG,65223000.0,37850000.0,7850000.0,30000000.0,NaN,NaN,NaN,...,NaN,351.40,13.21,NaN,NaN,10016.4787,3759.9565,0.14,0.11,0.42
3,Afghanistan,1964,AFG,65223000.0,37905000.0,7905000.0,30000000.0,NaN,NaN,NaN,...,NaN,335.61,11.63,NaN,NaN,10150.4670,3876.1390,0.14,0.11,0.40
4,Afghanistan,1965,AFG,65223000.0,37910000.0,7910000.0,30000000.0,NaN,NaN,NaN,...,NaN,377.70,12.59,NaN,NaN,10411.2382,3943.7140,0.14,0.11,0.37


## Processament de la variable d’ús de pesticides

Inicialment disposàvem de quatre variables:

- `Value_pesticide_agricultural_use_t`: ús total de pesticides (tones)
- `Value_6971_herbaceous_crops_ha`: superfície de cultius herbacis
- `Value_6974_tree-covered_areas_ha`: superfície d’àrees arborades
- `Value_6983_grassland_ha`: superfície de prats i herbassars


In [31]:
pesticide_vars = [
    "Value_pesticide_agricultural_use_t",
    "Value_6971_herbaceous_crops_ha",
    "Value_6974_tree-covered_areas_ha",
    "Value_6983_grassland_ha"
]
print("\nPercentatge de valors nuls:")
agri_country_year_df[pesticide_vars].isna().mean() * 100


Percentatge de valors nuls:


Value_pesticide_agricultural_use_t    42.808767
Value_6971_herbaceous_crops_ha        43.824664
Value_6974_tree-covered_areas_ha      43.824664
Value_6983_grassland_ha               43.824664
dtype: float64


Després d’analitzar-les, descartem les tres variables de superfície perquè les hectàrees ja estan controlades a través de les variables de *Land Use* que hem netejat prèviament, on el % de valors nul era molt inferior i ja aporta informació  suficient sobre tipus de camps i terres. Ens quedem únicament amb l’ús total de pesticides  manté consistència amb altres variables. Fem una interpolació anual dins de cada país per omplir valors buits, mantenint coherència temporal, fixant un mínim de 0, ja que no pot haver-hi valors negatius d’ús de pesticides.

In [32]:
df = agri_country_year_df.copy()


# 1. Renombrar la variable principal
df = df.rename(columns={
    "Value_pesticide_agricultural_use_t": "pesticide_agricultural_use_t"
})

col = "pesticide_agricultural_use_t"

# 2. Eliminar les altres 3 columnes
cols_to_drop = [
    "Value_6971_herbaceous_crops_ha",
    "Value_6974_tree-covered_areas_ha",
    "Value_6983_grassland_ha"
]

df = df.drop(columns=cols_to_drop, errors="ignore")

# 3. Interpolació anual per país
df = df.sort_values(["Country", "Year"])

df[col] = (
    df.groupby("Country")[col]
      .transform(lambda s: s.interpolate(method="linear"))
)

# 4. Fixar mínim 0
df[col] = df[col].clip(lower=0)


nan_count = df[col].isna().mean() * 100
print("Nombre de NaNs restants a pesticide_agricultural_use_t:", nan_count)
agri_country_year_df = df

Nombre de NaNs restants a pesticide_agricultural_use_t: 42.80876681403443


Això vol dir que els nuls venen de païssos en particular, en aquests casos, com el valor es tant alt, afegir una mediana ens faria crear soroll, per tant busquem un altre estratègia. Abans, acabem de tractar totes les variables per tenir una millor estratègia d'imputació.

## Variables climàtiques del dataset

Disposem de tres variables climàtiques fonamentals:

1. `prec_mm_per_year`: Indica la **precipitació anual total** expressada en mil·límetres.  

2. `mean_annual_tas_deg_celsius`: Representa la **temperatura mitjana anual** en graus Celsius.  

3. `surface_temperature_change_celsius`: Mesura el **canvi de temperatura respecte a un període base**, expressat en graus Celsius. És un indicador de l’impacte del canvi climàtic i de l’escalfament global sobre cada país.

Aquestes tres variables són informativament rellevants, no redundants amb altres variables del dataset i útils per a qualsevol model o anàlisi posterior.

Calculem el percentatge de valors nuls per entendre la seva qualitat i disponibilitat.


In [33]:
climate_cols = [
    "prec_mm_per_year",
    "mean_annual_tas_deg_celsius",
    "surface_temperature_change_celsius"
]

null_pct = agri_country_year_df[climate_cols].isna().mean() * 100

print("Percentatge de NaNs en les variables climàtiques:")
print(null_pct.round(2))


Percentatge de NaNs en les variables climàtiques:
prec_mm_per_year                      0.00
mean_annual_tas_deg_celsius           0.00
surface_temperature_change_celsius    5.41
dtype: float64


Fem interpolació temporal per país per veure si els NaNs de `surface_temperature_change_celsius` són “forats” puntuals dins d’una sèrie temporal.

In [34]:
df = agri_country_year_df.copy()

col = "surface_temperature_change_celsius"

# Interpolació temporal per país
df = df.sort_values(["Country", "Year"])
df[col] = df.groupby("Country")[col].transform(lambda s: s.interpolate(method="linear"))

# Percentatge total de nuls després de la interpolació
print("Percentatge total de NaNs després:", (df[col].isna().mean() * 100).round(2), "%")

# Percentatge de NaNs per ANY
nan_by_year = (
    df.groupby("Year")[col]
      .apply(lambda x: x.isna().mean() * 100)
      .round(2)
)

print("\nPercentatge de NaNs per any:")
print(nan_by_year)

Percentatge total de NaNs després: 4.18 %

Percentatge de NaNs per any:
Year
1961    7.81
1962    7.03
1963    6.98
1964    6.87
1965    6.11
        ... 
2019    2.91
2020    2.91
2021    2.91
2022    2.93
2023    2.93
Name: surface_temperature_change_celsius, Length: 63, dtype: float64


Veiem que encara tenim valors nuls, per tant imputem.

### Imputació de `surface_temperature_change_celsius`

`surface_temperature_change_celsius` encara presenta un 4.18% de valors nuls després d’intentar interpolació temporal per país.  
Aquest valor continua sent significatiu i pot limitar la qualitat de les visualitzacions que utilitzin informació climàtica.

Per aquest motiu, aplicarem una imputació basada en **KNN Imputer**, utilitzant informació espacial i climàtica addicional.

Per imputar els valors faltants utilitzarem quatre variables fortament relacionades:

1. Latitud del país 
2. Longitud del país
3. `prec_mm_per_year` — precipitació anual
4. `mean_annual_tas_deg_celsius` — temperatura mitjana anual

Mantindrem a més la latitud i longitud dels païssos ja que poden ser utils en les visualitzacions. A partir del codi de país `ISO3`, obtenim el centroid geogràfic de cada país (latitud i longitud) utilitzant el shapefile global de `Natural Earth` via GeoPandas. Això ens proporciona una localització aproximada del “centre” de cada país. https://developers.google.com/public-data/docs/canonical/countries_csv?hl=es-419

Aquest dataset proporciona un punt centrat per país. Per corregir inconsistències:

- S’ha eliminat **Congo (RDC)** i mantingut **Congo**, renombrat a “Congo”.
- S’ha afegit **South Sudan** manualment amb les coordenades oficials.


Aquests quatre predictores permeten capturar la variació espacial de la temperatura (latitud és especialment important), patrons regionals de precipitació i correlació directa entre temperatura mitjana i anomalies.

Les variables seran normalitzades temporalment per al KNN (z-score), però el dataframe original es mantindrà sense canviar.

Abans d'aplicar la imputació KNN, es van investigar altres imputacions més sofisticades, arrivant a *kriging*, que és un mètode geoestadístic molt potent per a interpolació espacial, basat en variograms i correlació espacial contínua.  
Documentació del mètode:  
https://en.wikipedia.org/wiki/Kriging

Tot i ser el mètode ideal per interpolar valors climàtics en l’espai, NO és aplicable
a aquest dataset perquè necessita molts punts d’observació amb coordenades reals (per exemple, estacions meteorològiques o cel·les d’una malla regular). En el nostre cas només tenim un punt per país (latitud/longitud del centre del país) i amb un únic punt per unitat, no es pot estimar un variograma ni calcular la dependència espacial.

Per tant, la interpolació espacial avançada no és viable aquí.

In [35]:
# 1. Còpia del dataframe original
df = agri_country_year_df.copy()


# 2. Importació del dataset de Google amb latitud i longitud
url = "https://raw.githubusercontent.com/google/dspl/master/samples/google/canonical/countries.csv"
coords = pd.read_csv(url, dtype=str)

coords["latitude"] = coords["latitude"].astype(float)
coords["longitude"] = coords["longitude"].astype(float)

# 3. Correccions manuals

coords = coords[coords["name"] != "Congo (RDC)"]

# Renombrar a "Congo"
coords.loc[coords["name"] == "Congo [Republic]", "name"] = "Democratic Republic of the Congo"
coords.loc[coords["name"] == "Congo [DRC]", "name"] = "Congo"

# Afegir South Sudan manualment
south_sudan_row = pd.DataFrame({
    "country": ["SS"],
    "latitude": [4.8539],
    "longitude": [31.5825],
    "name": ["South Sudan"]
})
coords = pd.concat([coords, south_sudan_row], ignore_index=True)

# Convertir 'name' : ISO3
coords["ISO3"] = coco.convert(coords["name"], to="ISO3")

# 4. Selecció de columnes necessàries per a merge
coords_clean = coords[["ISO3", "latitude", "longitude"]]

# 5. Merge per ISO3 amb el dataframe principal
df = df.merge(coords_clean, on="ISO3", how="left")

print("Països sense latitud:", df[df["latitude"].isna()]["Country"].unique())
print("Països sense longitud:", df[df["longitude"].isna()]["Country"].unique())

# 6. Definició de la variable a imputar i predictors per al KNN
target = "surface_temperature_change_celsius"

predictors = [
    "latitude",
    "longitude",
    "prec_mm_per_year",
    "mean_annual_tas_deg_celsius",
    target
]

# 7. Construcció del subdataframe només amb les predictores
knn_df = df[predictors].copy()

# 8. Normalització (escala z-score) de les variables per KNN
scaler = StandardScaler()
knn_scaled = scaler.fit_transform(knn_df)

# 9. Imputació amb KNN (k=5 veïns més propers)
imputer = KNNImputer(n_neighbors=5)
knn_imputed = imputer.fit_transform(knn_scaled)

# 10. Inversió de l’escalat (desnormalitzar)
knn_inverse = scaler.inverse_transform(knn_imputed)

# 11. Substitució de la variable imputada al dataframe original
df[target] = knn_inverse[:, predictors.index(target)]

# 12. Limitació de les anomalies de temperatura a un rang raonable [-5, 5]
df[target] = df[target].clip(lower=-5, upper=5)

# 13. Comprovació final
print("NaNs restants després KNN:", df[target].isna().mean())

# 14. Guardar resultat
agri_country_year_df = df


Netherlands Antilles not found in regex


Països sense latitud: []
Països sense longitud: []
NaNs restants després KNN: 0.0


Ara que ja tenim més variables, recordem quines variables ens queden amb valors nuls.

In [36]:
nulls_df = (
    agri_country_year_df.isna()
        .mean()
        .reset_index()
        .rename(columns={'index': 'column', 0: 'pct_nulls'})
)

# Filtrar solo columnas con > 0% nulos
nulls_df = nulls_df[nulls_df['pct_nulls'] > 0]

# Convertir a porcentaje y ordenar
nulls_df['pct_nulls'] = nulls_df['pct_nulls'] * 100
nulls_df = nulls_df.sort_values('pct_nulls', ascending=False).reset_index(drop=True)

display(nulls_df)

,column,pct_nulls
0,female_share_of_employment_in_agriculture,45.592791
1,total_share_of_employment_in_agriculture,45.592791
2,pesticide_agricultural_use_t,42.720360
3,agriculture_annual_growth_percent,28.151694
4,agriculture_share_gdp_percent,22.331738


## Imputació robusta del PIB i de les variables econòmiques agrícoles

Per tal de completar les variables econòmiques relacionades amb l'agricultura (`agriculture_share_gdp_percent` i `agriculture_annual_growth_percent`), primer és necessari obtenir una estimació fiable del **PIB total per país i any**, ja que totes aquestes magnituds estan directament relacionades amb la mida i l’evolució de l’economia de cada país.

Disposem d’un dataset extern de PIB: `../data/public_data/flat-ui__data-Tue_Dec_02_2025.csv` trobat a: https://datahub.io/core/gdp que podem utilitzar ja que tenen llicència d'open data.

Aquest dataset té valors per tots els països del món, però no cobreix tots els anys. Això vol dir que per completar la sèrie temporal cal aplicar
una imputació controlada.

Per imputar els valors faltants de PIB seguint un procés en dues fases, una interpolació temporal per país, que s'aplica `interpolate(method="linear")` dins de cada país. Això captura la tendència temporal de l’economia. Ajust basat en la població (GDP per càpita estable). Quan un país té molt pocs valors històrics i la interpolació lineal no és suficient, farem una regresió lineal sobre les diferències per inputar els valors faltants

De manera que, si hi ha GDP per càpita disponible per algun any, podem aproximar els anys intermedis. En absència de GDPpc, fem servir la relació entre població i PIB dels anys observats.


### Imputació de les variables econòmiques agrícoles

Un cop tenim el PIB complet, imputem:

- `agriculture_share_gdp_percent`  
- `agriculture_annual_growth_percent`

Utilitzant un **KNN Imputer** amb totes les variables rellevants disponibles,
incloent:

- PIB imputat  
- població  
- ús del sòl  
- emissions  
- fertilitzants  
- clima  
- geolocalització (latitud/longitud)

KNN és adequat perquè permet inferir valors a partir de països "similars" en estructura econòmica, clima i activitat agrícola.

Les variables es normalitzen temporalment per al KNN, però el dataframe original no es normalitza, només el conjunt auxiliar.

In [37]:
# Funció per imputar GDP_usd per país amb regressió de diferències
def impute_gdp_with_diff_regression(subdf: pd.DataFrame) -> pd.DataFrame:

    subdf = subdf.sort_values("Year").copy()
    y = subdf["GDP_usd"].values.astype(float)
    years = subdf["Year"].values.astype(int)

    # Si totes les dades són NaN : posem 0
    if np.isnan(y).all():
        subdf["GDP_usd"] = 0
        return subdf

    # Índexs amb dades vàlides
    mask = ~np.isnan(y)

    # Si només hi ha 1 dada : extrapolem constant
    if mask.sum() == 1:
        subdf["GDP_usd"] = y[mask][0]
        return subdf

    # Dades vàlides
    valid_years = years[mask]
    valid_values = y[mask]

    # Diferències observades A(GDP_usd)
    diff_years = valid_years[1:]
    diff_values = valid_values[1:] - valid_values[:-1]

    # Regressió GDP_usd ~ Year
    model = LinearRegression()
    model.fit(diff_years.reshape(-1,1), diff_values)

    # Predicció diferencial per tots els anys
    predicted_diff = model.predict(years.reshape(-1,1))

    # Reconstrucció de la sèrie
    start_index = np.where(mask)[0][0]
    gdp_series = np.zeros_like(y)
    gdp_series[start_index] = valid_values[0]

    # Cap endavant
    for i in range(start_index+1, len(years)):
        gdp_series[i] = gdp_series[i-1] + predicted_diff[i]

    # Cap enrere
    for i in range(start_index-1, -1, -1):
        gdp_series[i] = gdp_series[i+1] - predicted_diff[i]

    # Evitar negatius
    gdp_series = np.clip(gdp_series, 0, None)

    subdf["GDP_usd"] = gdp_series

    return subdf
    

# Inici de la transformació
df = agri_country_year_df.copy()


# Carregar dataset extern GDP
gdp_path = "../data/public_data/flat-ui__data-Tue_Dec_02_2025.csv"
gdp_raw = pd.read_csv(gdp_path)

# Filtrar entrades que no són països segons Coco
# Aquesta llista s'ha generat executant coco.convert per tot el dataframe i 
# revisant els valors únics que apareixen com a NaN o buits a 'Country Name'
no_country_values = [
    "Africa Eastern and Southern",
    "Africa Western and Central",
    "Arab World",
    "Caribbean small states",
    "Central Europe and the Baltics",
    "Channel Islands",
    "IBRD only",
    "IDA blend",
    "IDA only",
    "IDA total",
    "IDA & IBRD total",
    "Early-demographic dividend",
    "Heavily indebted poor countries (HIPC)",
    "Latin America & Caribbean",
    "Least developed countries: UN classification",
    "East Asia & Pacific",
    "Europe & Central Asia",
    "European Union",
    "Middle East & North Africa",
    "Sub-Saharan Africa",
    "Upper middle income",
    "High income",
    "Fragile and conflict affected situations",
    "Low & middle income",
    "Late-demographic dividend",
    "Low income",
    "Lower middle income",
    "Middle income",
    "North America",
    "OECD members",
    "Pacific island small states",
    "Post-demographic dividend",
    "Pre-demographic dividend",
    "Small states",
    "South Asia",
    "Upper middle income",
    "Euro area",
    "World",
    "Latin America & the Caribbean",
    "Other small states"
]
prefixes = tuple(s.lower() for s in no_country_values)
mask = gdp_raw["Country Name"].str.lower().str.startswith(prefixes)
gdp_raw = gdp_raw[~mask].copy()


gdp_raw["ISO3"] = coco.convert(gdp_raw["Country Name"], to="ISO3")
gdp_raw = gdp_raw.rename(columns={"Value": "GDP_usd"})

print("Preview del dataset GDP_usd extern:")
display(gdp_raw.head())

# Merge amb el dataframe principal
df = df.merge(
    gdp_raw[["ISO3", "Year", "GDP_usd"]],
    on=["ISO3","Year"],
    how="left"
)

# Aplicar la imputació país per país
df = (
    df.groupby("ISO3", group_keys=False)
      .apply(impute_gdp_with_diff_regression)
)

print("NaNs restants a GDP_usd després de regressió diferencial:", df["GDP_usd"].isna().mean()*100, "%")

# Guardar de nou
agri_country_year_df = df

print("Imputació GDP_usd completada.")

Preview del dataset GDP_usd extern:


,Country Name,Country Code,Year,GDP_usd,ISO3
0,Afghanistan,AFG,2000,3.521418e+09,AFG
1,Afghanistan,AFG,2001,2.813572e+09,AFG
2,Afghanistan,AFG,2002,3.825701e+09,AFG
3,Afghanistan,AFG,2003,4.520947e+09,AFG
4,Afghanistan,AFG,2004,5.224897e+09,AFG


NaNs restants a GDP_usd després de regressió diferencial: 0.0 %
Imputació GDP_usd completada.


/var/folders/x5/22k124b128j49p973rsrf6hc0000gn/T/ipykernel_64248/2932075251.py:131: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(impute_gdp_with_diff_regression)


No hi ha problemes amb els Countries no trobats, perquè efectivament no són païssos, si no agrupacions més de païssos que no necessitem.


In [38]:
# Passem al a impotació de les variables econòmiques
df = agri_country_year_df.copy()

# 1. Variables a imputar
targets = [
    "agriculture_share_gdp_percent",
    "agriculture_annual_growth_percent",
]

# 2. Predictores rellevants
predictors = [
    "GDP_usd",
    "total_population_No",
    "land_area_ha",
    "agricultural_land_ha",
    "cropland_ha",
    "emissions_total_agriculture_CO2eq_AR5_kt",
    "latitude",
    "longitude",
] + targets

knn_df = df[predictors].copy()


# 3. Normalització
scaler = StandardScaler()
X_scaled = scaler.fit_transform(knn_df)

# 4. KNN Imputer
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(X_scaled)

# 5. Desnormalització
X_final = scaler.inverse_transform(X_imputed)

df_imputed = df.copy()
for t in targets:
    df_imputed[t] = X_final[:, predictors.index(t)]

# 6. Clips
df_imputed["agriculture_share_gdp_percent"] = df_imputed["agriculture_share_gdp_percent"].clip(0, 100)
df_imputed["agriculture_annual_growth_percent"] = df_imputed["agriculture_annual_growth_percent"].clip(-100, 100)

print("NaNs finals després d'imputar:")
for t in targets:
    print(t, df_imputed[t].isna().mean(), "%")

agri_country_year_df = df_imputed


NaNs finals després d'imputar:
agriculture_share_gdp_percent 0.0 %
agriculture_annual_growth_percent 0.0 %


### Impurtació de `pesticide_agricultural_use_t`

En aquesta última etapa imputem els valors nuls de la variable `pesticide_agricultural_use_t`. Aquesta variable expressa l’ús total de pesticides en tones i és coherent mantenir-la al dataset, ja que està relacionada amb productivitat agrícola, intensitat d’ús del sòl i pressió sobre els ecosistemes.

Per a la imputació utilitzem únicament les variables que tenen sentit com a predictores. Seleccionem variables demogràfiques com la població total, variables d’ús del sòl com les hectàrees agrícoles o de conreu, variables de clima com la precipitació i la temperatura mitjana i variables d’emissions derivades de l’agricultura. Aquestes característiques permeten definir bé la magnitud de l’activitat agrícola d’un país i són fortes candidates a estar correlacionades amb l’ús de pesticides.

Fem servir KNN Imputer perquè és un mètode robust per a variables d’escala diferent i aprofita similituds multidimensionals entre països i anys. Només normalitzem les variables dins del procés d’imputació i després desfem l’escalat perquè el dataframe final mantingui totes les variables en la seva escala original.

Una vegada imputada la variable, es converteix de tones a quilotones (kt) perquè totes les altres variables d’emissions del dataset estan expressades en kt i cal mantenir coherència d’unitats.

In [39]:
df = agri_country_year_df.copy()

target = "pesticide_agricultural_use_t"

predictors = [
    "land_area_ha",
    "agricultural_land_ha",
    "cropland_ha",
    "permanent_meadows_and_pastures_ha",
    "emissions_soils_CO2eq_AR5_kt",
    "emissions_rice_cultivation_agricultural_CO2eq_AR5_kt",
    "emissions_synthetic_fertilizers_CO2eq_AR5_kt",
    "emissions_crop_residues_CO2eq_AR5_kt",
    "total_population_No",
    "female_population_%",
    "rural_population_%",
    "surface_temperature_change_celsius",
    "agriculture_share_gdp_percent",
    "agriculture_annual_growth_percent",
    "prec_mm_per_year",
    "mean_annual_tas_deg_celsius",
    "emissions_total_livestock_CO2eq_AR5_kt",
    "emissions_total_agriculture_CO2eq_AR5_kt",
    "fertilizers_per_area_of_cropland_kg_ha",
    "fertilizers_per_capita_kg_cap",
    "fertilizers_per_value_of_agri_prod_g_Intdollar",
    "latitude",
    "longitude",
    "GDP_usd",
    target
]

knn_df = df[predictors].copy()

scaler = StandardScaler()
knn_scaled = scaler.fit_transform(knn_df)

imputer = KNNImputer(n_neighbors=5)
knn_imputed = imputer.fit_transform(knn_scaled)

knn_inverse = scaler.inverse_transform(knn_imputed)

df[target] = knn_inverse[:, predictors.index(target)]

df[target] = df[target].clip(lower=0)

df["pesticide_agricultural_use_kt"] = df[target] / 1000

print("vals nuls restants:", df[target].isna().mean()*100, "%")

agri_country_year_df = df

vals nuls restants: 0.0 %


### Imputació de les variables de participació laboral femenina i total en l'agricultura

Les variables `female_share_of_employment_in_agriculture` i `total_share_of_employment_in_agriculture` representen el percentatge de la població ocupada en el sector agrícola, desagregada per gènere i també en total. Aquestes variables solen dependre fortament del temps (canvis estructurals de cada país), però també tenen components espacials i agrícoles que poden ajudar en la imputació.

Per aquest motiu, s’aplica una estratègia d’imputació en tres passos.

1. Interpolació temporal per país: Si un país té valors en diversos anys, es pot reconstruir la tendència temporal mitjançant interpolació lineal. Aquest és el millor mètode quan hi ha dades suficients abans i després del buit, ja que conserva el patró temporal propi de cada país.

2. Regressió sobre la diferència anual per als casos on la interpolació no és possible: Hi ha casos en què la interpolació no pot funcionar, com ara:
- el primer valor conegut apareix molt tard
- només hi ha un o dos punts dispersos
- no hi ha valors anteriors per interpolar cap enrere

En aquests casos s’estima la sèrie completa mitjançant una regressió sobre les diferències anuals del percentatge, permetent reconstruir tota la trajectòria temporal. Els valors imputats es limiten a l’interval [0 , 100], ja que tota la sèrie representa percentatges.

3. Imputació final basada en característiques espacials i agràries: Els valors restants s’imputen utilitzant variables que tenen correlació amb l’estructura laboral agrícola d’un país, com la latitud, la longitud, la superfície agrícola, la població, l’ús de fertilitzants o el clima. Per a aquest darrer pas s’aplica un model KNN amb variables normalitzades, assegurant que els països geogràficament i agronòmicament similars s’utilitzen com a veïns en la imputació.

Finalment s’aplica una retallada perquè tots els valors quedin dins l’interval [0, 100] i s’assegura que no quedin valors perduts després del procés.

In [40]:
# Es fa una còpia del dataframe principal amb les dades per any i país
df = agri_country_year_df.copy()

# Variables a imputar: participació laboral femenina i total a l'agricultura
targets = [
    "female_share_of_employment_in_agriculture",
    "total_share_of_employment_in_agriculture"
]

# Funció d'imputació per regressió sobre la diferència anual
def impute_with_diff_regression(subdf: pd.DataFrame, col: str) -> pd.DataFrame:
    # S'ordena per any i es fa còpia
    s = subdf.sort_values("Year").copy()
    y = s[col].values.astype(float)
    years = s["Year"].values.astype(int)

    # Si tots els valors són NaN, es retorna sense canvis
    if np.isnan(y).all():
        s[col] = np.nan
        return s

    mask = ~np.isnan(y)

    # Si només hi ha un valor disponible, s'omple tota la sèrie amb aquest valor (acotant-lo a [0,100])
    if mask.sum() == 1:
        s[col] = np.clip(y[mask][0], 0, 100)
        return s

    # S'extreuen els anys i valors vàlids
    valid_years = years[mask]
    valid_values = y[mask]

    # Es calculen les diferències any a any
    diff_y = valid_values[1:] - valid_values[:-1]
    diff_x = valid_years[1:]

    # S'entrena una regressió lineal per predir la diferència anual
    model = LinearRegression()
    model.fit(diff_x.reshape(-1,1), diff_y)

    # Es prediu la diferència per tots els anys de la sèrie
    predicted_diff = model.predict(years.reshape(-1,1))

    # S'estableix el primer índex amb dada coneguda i es reconstrueix la sèrie sencera
    start_idx = np.where(mask)[0][0]
    reconstructed = np.zeros_like(y)
    reconstructed[start_idx] = valid_values[0]

    # Reconstruïm cap endavant
    for i in range(start_idx+1, len(years)):
        reconstructed[i] = reconstructed[i-1] + predicted_diff[i]

    # Reconstruïm cap enrere
    for i in range(start_idx-1, -1, -1):
        reconstructed[i] = reconstructed[i+1] - predicted_diff[i]

    # Retallem valors a l'interval [0, 100]
    reconstructed = np.clip(reconstructed, 0, 100)
    s[col] = reconstructed
    return s

# Pas 1 i 2: S'aplica la funció d'interpolació/regressió per cada país i variable
for col in targets:
    df = (
        df.groupby("ISO3")
          .apply(lambda g: impute_with_diff_regression(g, col))
          .reset_index(drop=True)
    )

# Variables utilitzades per la imputació KNN (pas 3)
knn_predictors = [
    "latitude",
    "longitude",
    "agricultural_land_ha",
    "cropland_ha",
    "total_population_No",
    "surface_temperature_change_celsius",
    "prec_mm_per_year",
    "mean_annual_tas_deg_celsius",
    "fertilizers_per_capita_kg_cap",
    "fertilizers_per_area_of_cropland_kg_ha",
    "emissions_total_agriculture_CO2eq_AR5_kt"
]

# S'aplica el model KNN per imputar la resta de valors
for col in targets:
    # Es construeix un dataframe auxiliar només amb les predictores i la variable diana
    aux = df[knn_predictors + [col]].copy()

    # S'escalen les variables per igualar la importància
    scaler = StandardScaler()
    aux_scaled = scaler.fit_transform(aux)

    # S'imputa amb KNN (5 veïns)
    imputer = KNNImputer(n_neighbors=5)
    aux_imputed = imputer.fit_transform(aux_scaled)

    # Es retorna als valors originals la variable imputada
    aux_inverse = scaler.inverse_transform(aux_imputed)

    # Es recupera la columna imputada i es retalla [0,100]
    df[col] = aux_inverse[:, aux.columns.get_loc(col)]
    df[col] = df[col].clip(0, 100)

# S'actualitza el dataframe principal amb les noves imputacions
agri_country_year_df = df

# Output del resum de valors (comprovació que no resten NaNs)
print("Imputació completa sense NaNs restants:")
for col in targets:
    print(col, ":", df[col].isna().sum())

/var/folders/x5/22k124b128j49p973rsrf6hc0000gn/T/ipykernel_64248/2568610675.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: impute_with_diff_regression(g, col))
/var/folders/x5/22k124b128j49p973rsrf6hc0000gn/T/ipykernel_64248/2568610675.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: impute_with_diff_regression(g, col))


Imputació completa sense NaNs restants:
female_share_of_employment_in_agriculture : 0
total_share_of_employment_in_agriculture : 0


Podriem solucionar els warning amb `include_group=False`, però al fer-ho es perden algunes variables, així que ho mantenim per aquesta pràctica.

Ara que ja tenim tot el data frame estudiat i totes les variables imputades, el guardem per la posterior pràctica i pasem a l'anàlisi del següent data frame.

In [41]:
agri_country_year_df.to_csv("../data/visualization_input_processed/agri_country_year_df_processed.csv", index=False)

## Segon dataset: agri_production_and_prices_df

In [42]:
agri_production_and_prices_df = pd.read_csv("../data/visualization_input/agri_production_and_prices_df.csv")
print("Primeres files del dataset:")
display(agri_production_and_prices_df.head())
print("\nColumnes del dataset:")
print(agri_production_and_prices_df.columns.tolist())

Primeres files del dataset:


,ISO3,Area Code,Area Code (M49),Area,Year Code,Year,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Value,Flag,Flag_Description,Value_LCU/tonne,Value_SLC/tonne,Value_USD/tonne
0,AFG,2,'004,Afghanistan,1961,1961,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,A,Official figure,NaN,NaN,NaN
1,AFG,2,'004,Afghanistan,1962,1962,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,A,Official figure,NaN,NaN,NaN
2,AFG,2,'004,Afghanistan,1963,1963,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,A,Official figure,NaN,NaN,NaN
3,AFG,2,'004,Afghanistan,1964,1964,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,A,Official figure,NaN,NaN,NaN
4,AFG,2,'004,Afghanistan,1965,1965,221,'01371,"Almonds, in shell",5312,Area harvested,ha,0.0,A,Official figure,NaN,NaN,NaN



Columnes del dataset:
['ISO3', 'Area Code', 'Area Code (M49)', 'Area', 'Year Code', 'Year', 'Item Code', 'Item Code (CPC)', 'Item', 'Element Code', 'Element', 'Unit', 'Value', 'Flag', 'Flag_Description', 'Value_LCU/tonne', 'Value_SLC/tonne', 'Value_USD/tonne']


In [43]:
display(agri_production_and_prices_df.groupby('Element')['Item'].unique())

Element
Area harvested                   [Almonds, in shell, Anise, badian, coriander, ...
Laying                           [Hen eggs in shell, fresh, Eggs Primary, Eggs ...
Milk Animals                     [Raw milk of camel, Raw milk of cattle, Raw mi...
Producing Animals/Slaughtered    [Cattle fat, unrendered, Edible offal of cattl...
Production                       [Almonds, in shell, Anise, badian, coriander, ...
Stocks                           [Asses, Camels, Cattle, Chickens, Goats, Horse...
Yield                            [Almonds, in shell, Anise, badian, coriander, ...
Yield/Carcass Weight             [Hen eggs in shell, fresh, Meat of camels, fre...
Name: Item, dtype: object

## Exclusió de registres relacionats amb la ramaderia

L’objectiu d’aquest dataset és centrar l’anàlisi en la producció agrícola, rendiments, superfícies i preus associats als cultius. Per aquest motiu, els registres relacionats amb la ramaderia no aporten informació rellevant per al propòsit principal i, a més, introdueixen heterogeneïtat en les unitats, en els elements i en la interpretació de les variables.

Les categories de `Element` que corresponen clarament a activitats ramaderes, i que per tant no són pertinents per a una anàlisi agrícola, són:

- *Laying* : Els items són tots relacionats amb ous.
- *Milk Animals*  : Els items són relacionats amb llet.
- *Producing Animals/Slaughtered*   : Els items són relacionats amb carns i elements de sacrificies.
- *Yield/Carcass Weight*  : Associats a animals vius
- *Stocks*: En aquest dataset, associats a animals vius.

Aquestes categories estan vinculades a productes animals (ous, llet, bestiar viu, carn, canal), i no a cultius.

Per preservar coherència i qualitat analítica, eliminem tots els registres associats a aquests elements per mantenir exclusivament informació agrícola (cultius, superfícies, rendiments vegetals). Aquesta depuració simplifica el dataset i permet una anàlisi més robusta i centrada en el sector agrícola estricte.


In [44]:
df = agri_production_and_prices_df.copy()

# Llista d'elements clarament ramaders
livestock_elements = [
    "Laying",
    "Milk Animals",
    "Producing Animals/Slaughtered",
    "Yield/Carcass Weight",
    "Stocks"  # en aquest dataset fa referència a animals vius
]

# Filtrar i mantenir només elements agrícoles
df_agri = df[~df["Element"].isin(livestock_elements)].copy()

print("Registres inicials:", len(df))
print("Registres després de retirar ramaderia:", len(df_agri))
print("Percentatge eliminat:", round((1 - len(df_agri)/len(df)) * 100, 2), "%")

# Guardar resultat
agri_production_and_prices_df = df_agri

# Mostrar els elements finals per validar
print("\nElements restants:")
display(agri_production_and_prices_df["Element"].unique())

Registres inicials: 3126610
Registres després de retirar ramaderia: 2528600
Percentatge eliminat: 19.13 %

Elements restants:


array(['Area harvested', 'Yield', 'Production'], dtype=object)

Estudiem ara si els 3 elements tenen els mateixos items

In [45]:
df = agri_production_and_prices_df.copy()

elements_keep = ["Area harvested", "Yield", "Production"]

# Diccionari: element: conjunt d’items
items_by_element = {
    el: set(df[df["Element"] == el]["Item"].unique())
    for el in elements_keep
}

for e1, e2 in combinations(elements_keep, 2):
    items1 = items_by_element[e1]
    items2 = items_by_element[e2]
    
    print(f"\nComparació entre: {e1}  ↔  {e2}")
    print("Items només a", e1, ":", items1 - items2)
    print("Items només a", e2, ":", items2 - items1)


Comparació entre: Area harvested  ↔  Yield
Items només a Area harvested : {'Coir, raw'}
Items només a Yield : {'Eggs from other birds in shell, fresh, n.e.c.', 'Hen eggs in shell, fresh'}

Comparació entre: Area harvested  ↔  Production
Items només a Area harvested : set()
Items només a Production : {'Meat of pigeons and other birds n.e.c., fresh, chilled or frozen', 'Other meat of mammals, fresh or chilled', 'Edible offal of sheep, fresh, chilled or frozen', 'Oil of sesame seed', 'Edible offals of horses and other equines,  fresh, chilled or frozen', 'Whey, condensed', 'Shorn wool, greasy, including fleece-washed shorn wool', 'Raw hides and skins of goats or kids', 'Meat of other domestic camelids, fresh or chilled', 'Meat of asses, fresh or chilled', 'Meat of sheep, fresh or chilled', 'Beeswax', 'Cheese from whole cow milk', 'Oil of maize', 'Palm oil', 'Brazil nuts, in shell', 'Eggs from other birds in shell, fresh, n.e.c.', 'Meat of turkeys, fresh or chilled', 'Buffalo fat, unrende

## Filtrat final d’ítems per centrar-nos en agricultura de conreu

Un cop eliminades les sèries clarament ramaderes (animals, llet, carn, greixos, ous, mel…), ens quedem amb tres elements principals rellevants per a l’agricultura de conreu:

- Area harvested  
- Yield  
- Production  

L’objectiu és treballar amb cultius per als quals disposem d’informació coherent i completa: superfície, rendiment i producció del mateix cultiu. Per això, estudiem si els *ITEM* associats a cada element coincideixen.

En comparar els conjunts d’ítems veiem que:

- Alguns ítems només existeixen a *Area harvested* o *Yield*, casos puntuals o productes molt específics.
- Però sobretot, hi ha una llista molt extensa d’ítems presents només a *Production*, la majoria dels quals corresponen a:
  - productes d’origen animal,
  - llet i derivats,
  - greixos animals,
  - ous, mel i productes processats,
  - olis, derivats lactis, carn fresca o processada, etc.

Aquests productes, tot i formar part de la cadena agroalimentària, no pertanyen a l’agricultura de conreu i no disposen d’una correspondència clara amb *Area harvested* i *Yield*.

Per aquest motiu, obtenim el conjunt d’ítems que apareixen en cadascun dels tres elements (Area harvested, Yield, Production) i considerem aquesta intersecció com el nucli dur de cultius coherents, ja que per a ells tenim superfície, producció i rendiment (pot ser s'ens escapa algun element, però com hem repetit reiterats cops, l'objectiu és poder tenir visualitzacions amb sentit, i no uns resultats de agricultura acurats). 

Eliminem del dataset tots els registres amb *ITEM* que no formen part d’aquesta intersecció. Això garanteix que treballem exclusivament amb cultius, sense hardcodejar llistes manuals, i aprofitant la informació real del dataset per determinar quins ítems són agrícoles de forma objectiva.


In [46]:
df = agri_production_and_prices_df.copy()

# 1. Calcular el conjunt d’ítems per a cada element
items_per_element = {}
for elem in df['Element'].unique():
    items_per_element[elem] = set(
        df.loc[df['Element'] == elem, 'Item'].unique()
    )

# 2. Intersecció d’ítems comuns entre tots els elements
items_comuns = set.intersection(*items_per_element.values())

print("Nombre total d’ítems diferents abans del filtratge:", 
      df['Item'].nunique())
print("Nombre d’ítems comuns a Area harvested, Yield i Production:", 
      len(items_comuns))

# 3. Ítems que es descarten (no apareixen als tres elements)
items_totals = set(df['Item'].unique())
items_descartar = items_totals - items_comuns

print("Nombre d’ítems que es descarten:", len(items_descartar))

# 4. Filtrar el dataframe per quedar-nos només amb els ítems comuns
df_filtrat = df[df['Item'].isin(items_comuns)].copy()

print("Nombre d’ítems després del filtratge:", 
      df_filtrat['Item'].nunique())
print("Nombre de files després del filtratge:", len(df_filtrat))

agri_production_and_prices_df = df_filtrat


Nombre total d’ítems diferents abans del filtratge: 281
Nombre d’ítems comuns a Area harvested, Yield i Production: 169
Nombre d’ítems que es descarten: 112
Nombre d’ítems després del filtratge: 169
Nombre de files després del filtratge: 1952472


Observem que continuem tenint pràcticament 2M de registres, més que suficient pel nostre objectiu. Ara, abans de continuar amb l'anàlisi:
1. Modificar els noms d'algunes columnes per que siguen més descriptives
2. Borrar les columnes que no siguen necessàries.
3. Analitzar els diferents valors que tenim a `Element` i `Flag_Description.
4. Analitzar els % de valors restantes de les files resultatns

In [47]:
df = agri_production_and_prices_df.copy()

# 1. Drop columns not needed
cols_to_drop = [
    'Area Code'
    , 'Area Code (M49)'
    , 'Year Code'
    , 'Item Code (CPC)'
    , 'Item Code'
    , 'Element Code'
    , 'Flag'
    , 'Flag_Description'
]
df = df.drop(columns=cols_to_drop)

# 2. Rename columns to be more descriptive
df = df.rename(columns={
    'Area': 'Country'
    , 'Item': 'Commodity'
    , 'Element': 'element_name'
    , 'Unit': 'unit'
    , 'Value': 'value'
    , 'Value_LCU/tonne': 'value_lcu_tonne'
    , 'Value_SLC/tonne': 'value_slc_tonne'
    , 'Value_USD/tonne': 'value_usd_tonne'
})

# 3. Inspect unique values in Element
unique_elements = df['element_name'].unique()

print("Unique Element values:", unique_elements)

# 4. Analitza el % de valors no nuls 
notna_means = df.isna().mean() * 100
print("Percentatge de valors no nuls per columna:")
print(notna_means[notna_means > 0])

agri_production_and_prices_df = df


Unique Element values: ['Area harvested' 'Yield' 'Production']
Percentatge de valors no nuls per columna:
value               6.603014
value_lcu_tonne    68.884471
value_slc_tonne    68.925803
value_usd_tonne    69.693087
dtype: float64


Observem que tenim pocs valors nuls a `value`, però molts als preus. Estudiem els porcentatges de valors nuls dels preus en funció de `Element`.

In [48]:
df = agri_production_and_prices_df.copy()

price_cols = ["value", "value_lcu_tonne", "value_slc_tonne", "value_usd_tonne"]

# Calcul del % de NaNs per Element i per cadascuna de les columnes de preu
nan_by_element = (
    df.groupby("element_name")[price_cols]
      .apply(lambda x: x.isna().mean() * 100)
      .round(2)
)

print("Percentatge de NaNs per Element i tipus de preu:")
display(nan_by_element)

print("Unitats per cada element:")
display(df.groupby('element_name')['unit'].unique())


Percentatge de NaNs per Element i tipus de preu:


,value,value_lcu_tonne,value_slc_tonne,value_usd_tonne
element_name,,,,
Area harvested,10.57,69.87,69.91,70.65
Production,8.32,70.23,70.27,71.00
Yield,0.00,66.16,66.21,67.05


Unitats per cada element:


element_name
Area harvested       [ha]
Production            [t]
Yield             [kg/ha]
Name: unit, dtype: object

## Tractament de les variables de preu i de les variables de informació de producció

En aquest punt hem calculat el percentatge de valors nuls per columna i element, i observem:

- `value` té molt pocs valors nuls (entre un 0 % i un 10 % segons l’element).
- Les columnes de preu `value_lcu_tonne`, `value_slc_tonne` i `value_usd_tonne` tenen aproximadament entre un 66 % i un 71 % de valors nuls per a tots els elements.

A més, l’objectiu principal d’aquesta anàlisi no és estudiar preus, sinó caracteritzar les trajectòries de producció agrícola: superfície sembrada, producció i rendiment per cultiu, país i any. A més, imputar una variable amb un ~70% de valors faltants és poc fiable, i més quan no tenim cap més infomració sobre cada producte particular per poder inputar el seu preu. Per tant, és més raonable conservar només la columna `value` com a magnitud física principal i eliminar les tres columnes de preu, que aporten poca informació i molta manca de dades.

Un cop comprovat també que, després de filtrar ramaderia, els tres elements restants (`Area harvested`, `Yield`, `Production`) comparteixen el mateix conjunt d’`item_name` (ara `Commodity`), té sentit reorganitzar el dataset en format ample per a cada combinació:

ISO3, Country, Year, Commodity

de manera que per a cada fila tinguem, en columnes diferents:

- la superfície recol·lectada (hectàrees),
- la producció (en quilotones per seguir la linea de l'anterior data frame),
- el rendiment (kg per hectàrea).

Concretament, passarem de files en format llarg com:

ISO3, Country, Year, Commodity, element_name, unit, value

a un format tipus:

ISO3, Country, Year, Commodity, area_harvested_ha, production_kt, yield_kg_ha

On:

- els registres amb element_name = "Area harvested" alimenten la columna `area_harvested_ha` (unitat ha),
- els registres amb element_name = "Production" alimenten la columna `production_kt`, convertint prèviament els valors de tones a quilotones dividint per 1000,
- els registres amb element_name = "Yield" alimenten la columna `yield_kg_ha` (unitat kg/ha).

Aquesta transformació ens deixa una taula molt més usable per a models i gràfics: una fila per cultiu, país i any, amb les tres magnituds bàsiques d’interès agrícola, sense redundàncies de files per element i sense soroll de columnes de preus altament incompletes.


In [49]:
df = agri_production_and_prices_df.copy()

# Eliminem columnes de preu, que tenen molts valors nuls
price_cols = [
    "value_lcu_tonne",
    "value_slc_tonne",
    "value_usd_tonne"
]
df = df.drop(columns=price_cols, errors="ignore")

# Pivotem el dataframe per passar d'format llarg a ample
#    Una fila per ISO3, Country, Year, Commodity, amb una columna per a cada element_name
pivot = (
    df
    .pivot_table(
        index=["ISO3", "Country", "Year", "Commodity"],
        columns="element_name",
        values="value",
        aggfunc="first"
    )
    .reset_index()
)

# Convertim la producció de tones a quilotones
if "Production" in pivot.columns:
    pivot["Production"] = pivot["Production"] / 1000.0

# 4. Renombrar columnes a noms més clars
rename_map = {}
if "Area harvested" in pivot.columns:
    rename_map["Area harvested"] = "area_harvested_ha"
if "Production" in pivot.columns:
    rename_map["Production"] = "production_kt"
if "Yield" in pivot.columns:
    rename_map["Yield"] = "yield_kg_ha"

pivot = pivot.rename(columns=rename_map)

print("Preview del dataframe en format ample:")
display(pivot.head())

agri_production_and_prices_df = pivot

Preview del dataframe en format ample:


element_name,ISO3,Country,Year,Commodity,area_harvested_ha,production_kt,yield_kg_ha
0,AFG,Afghanistan,1961,"Almonds, in shell",0.0,0.0,NaN
1,AFG,Afghanistan,1961,Apples,2220.0,15.1,6801.8
2,AFG,Afghanistan,1961,Apricots,4820.0,32.0,6639.0
3,AFG,Afghanistan,1961,Barley,350000.0,378.0,1080.0
4,AFG,Afghanistan,1961,Cantaloupes and other melons,3300.0,15.7,4757.6


In [50]:
# Un cop tenim el dataframe en format ample, podem estudiar els valors faltants per cada columna
df = agri_production_and_prices_df.copy()

# 1. Calcular el percentatge de valors nuls per columna
null_counts = df.isna().mean() * 100
display(null_counts[null_counts > 0])

element_name
area_harvested_ha    3.728711
production_kt        0.060749
yield_kg_ha          6.894803
dtype: float64

### Estratègia d’imputació per a les variables agrícoles (area_harvested_ha, production_kt, yield_kg_ha)

Després del pivotatge, les nostres tres variables principals presenten un nivell moderat de valors faltants:

- area_harvested_ha: ~3.7%
- production_kt: ~0.06%
- yield_kg_ha: ~6.9%

Aquests valors faltants poden venir per dos motius que no siguen error a les dades:
1) molts països no produeixen determinats cultius cada any,
2) alguns cultius tenen produccions intermitents o superfícies molt petites que poden aparèixer com a zero en determinats anys.

Per mantindre coherència econòmica i agronòmica, definim una estratègia d’imputació en tres passos:

1. Interpolació temporal per cada combinació (país + commodity): La producció, la superfície i el rendiment solen variar gradualment en el temps. Per tant, primer apliquem una interpolació lineal per anys a cada grup ISO3 + commodity.

2. Assignar 0 quan tot el vector d’un grup és només {0, NaN}: Si per a un país i un cultiu determinat, en totes les observacions només hi ha zeros i valors nuls, assumim que el cultiu no es produeix realment en aquell país. És un criteri fiable perquè evita imputar valors artificials allà on no hi ha agricultura real, encara que tenim el risc de posar 0 on realment só hi havia producció.

3. Imputació per regressió lineal només quan el grup té com a mínim dos anys amb valors positius. Per commodities reals, si després de la interpolació encara queden valors NaN, fem una regressió lineal simple basada en la tendència temporal.  Només s'aplica quan existeixen almenys dos punts >0, cosa que permet estimar una tendència. La predicció es limita a valors >= 0 per evitar resultats no físics.

Després d’aquest procés revisarem de nou si queden valors nuls.  En cas d’existir encara buits, considerarem utilitzar informació del primer dataframe agregat per aplicar una imputació més sofisticada (KNN o regressió multivariable).


In [51]:
# Imputació per regressió lineal temporal (si almenys 2 anys tenen valors > 0)
def regress_impute(subdf, col):
    y = subdf[col].values.astype(float)
    years = subdf["Year"].values.astype(float)

    mask_valid = (~np.isnan(y)) & (y > 0)

    if mask_valid.sum() < 2:
        return y

    model = LinearRegression()
    model.fit(years[mask_valid].reshape(-1, 1), y[mask_valid])

    y_pred = model.predict(years.reshape(-1, 1))
    y_imputed = np.where(np.isnan(y), y_pred, y)
    y_imputed = np.clip(y_imputed, 0, None)
    return y_imputed

# Assignem 0 quan un grup té només {0, NaN}
def all_zero_or_nan(series):
    vals = series.dropna().unique()
    return len(vals) == 1 and vals[0] == 0

# Processament de dades

df = agri_production_and_prices_df.copy()

vars_to_impute = ["area_harvested_ha", "production_kt", "yield_kg_ha"]

# Interpolació temporal per país + commodity
df = df.sort_values(["ISO3", "Commodity", "Year"])

for col in vars_to_impute:
    df[col] = (
        df.groupby(["ISO3", "Commodity"])[col]
          .transform(lambda s: s.interpolate(method="linear"))
    )

for (iso, com), subdf in df.groupby(["ISO3", "Commodity"]):
    for col in vars_to_impute:
        if all_zero_or_nan(subdf[col]):
            mask = (df["ISO3"] == iso) & (df["Commodity"] == com)
            df.loc[mask, col] = 0

df = df.sort_values(["ISO3", "Commodity", "Year"]).copy()

for col in vars_to_impute:
    df[col] = (
        df.groupby(["ISO3", "Commodity"])
          .apply(lambda g: pd.Series(regress_impute(g, col), index=g.index))
          .reset_index(level=[0,1], drop=True)
    )

remaining_nans = df[vars_to_impute].isna().mean() * 100
print("Percentatge de NaNs restants després d’imputar:")
print(remaining_nans)

agri_production_and_prices_df = df

/var/folders/x5/22k124b128j49p973rsrf6hc0000gn/T/ipykernel_64248/3631410497.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(regress_impute(g, col), index=g.index))
/var/folders/x5/22k124b128j49p973rsrf6hc0000gn/T/ipykernel_64248/3631410497.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(regress_impute(g, col), index=g.index))


Percentatge de NaNs restants després d’imputar:
element_name
area_harvested_ha    0.438205
production_kt        0.013770
yield_kg_ha          0.947203
dtype: float64


/var/folders/x5/22k124b128j49p973rsrf6hc0000gn/T/ipykernel_64248/3631410497.py:50: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(regress_impute(g, col), index=g.index))


Estudiem d'on venen els valors.

In [52]:
df = agri_production_and_prices_df.copy()

# Registres on TOTS tres valors són NaN
mask_all_three_nan = (
      df["area_harvested_ha"].isna()
    & df["production_kt"].isna()
    & df["yield_kg_ha"].isna()
)

df_all_nan = df[mask_all_three_nan]

print("Nombre de registres amb els TRES valors NaN:", len(df_all_nan))
display(df_all_nan[["ISO3", "Country", "Year", "Commodity"]].head(50))



Nombre de registres amb els TRES valors NaN: 0


element_name,ISO3,Country,Year,Commodity


### Imputació final basada en proporcions anuals per commodity

Després de la interpolació temporal país–commodity i de la regressió lineal per completar sèries amb prou informació, encara queden alguns valors nuls puntuals. Aquests casos no poden correspondre a buits temporals (ja resolts), de manera que la causa més probable és la manca d’informació parcial d’algun element mesurat en aquell any concret.

Per resoldre aquests casos, utilitzem una estratègia basada en proporcions anuals dins de cada any i commodity. Això s’explica perquèsi féssim la mitjana per país+commodity, aquest valor ja hauria estat interpolat o regressat. En canvi, dins d’un mateix any i per un mateix cultiu, la relació entre `area_harvested_ha`, `production_kt` i `yield_kg_ha` pot ser utilitzada per inputar el valors restants, tenint en compte que no hi ha cap registre on els 3 valors siguen nuls.

Així, per a cada registre \((c,p,t)\), on commodity \(c\), un pais \(p\) i un any \(t\), amb:
- \(A_{cpt}\): area_harvested_ha  
- \(P_{cpt}\): production_kt  
- \(Y_{cpt}\): yield_kg_ha  

Tindrem que, en primera instància podem considerar com aproximació que

1000*\frac{Y_{cpt}} approx P_{cpt} / A_{cpt} \cdot 

Si falta algun dels tres valors, com sempre, em vist que, almenys un està present , podem reconstruir-lo utilitzant les proporcions mitjanes del mateix commodity i mateix any.

Per un commodity \(k\) i any \(t\), definim:


\mu_A^{kt} = Mean(A_{cpt})_{c}


\mu_P^{kt} = Mean(P_{cpt})_{c}


\mu_Y^{kt} = Mean(Y_{cpt})_{c}


Aleshores, per exemple, si falta \(A_{cpt}\) però tenim \(P_{cpt}\):


A_{cpt} = \frac{P_{cpt}}{\mu_P^{kt}} \cdot \mu_A^{kt}


És a dir, assignem el valor que manté la mateixa proporció relativa que la resta de països aquell any.

Aquesta estratègia funciona perquè sempre hi ha almenys un valor no nul en cada registre, les mitjanes anuals per commodity són representatives i no estan afectades per la interpolació i garantim coherència dimensional i proporcional entre les tres variables.

Un cop aplicat el càlcul a les tres magnituds, revisem novament els valors nuls per confirmar que la imputació és completa.


In [53]:
df = agri_production_and_prices_df.copy()  # dataset ja filtrat i pivotat

cols = ["area_harvested_ha", "production_kt", "yield_kg_ha"]

# Calculem mitjanes per any + commodity
means = (
    df.groupby(["Year", "Commodity"])[cols]
      .mean()
      .rename(columns=lambda x: f"mean_{x}")
)

# Afegim les mitjanes al dataframe
df = df.merge(
    means,
    on=["Year", "Commodity"],
    how="left"
)

# Definim la funció per imputar segons proporcions anuals per commodity
def impute_row(row):
    A, P, Y = row["area_harvested_ha"], row["production_kt"], row["yield_kg_ha"]
    mA, mP, mY = row["mean_area_harvested_ha"], row["mean_production_kt"], row["mean_yield_kg_ha"]

    #  1) imputem a (superfície)
    if np.isnan(A):
        # 1.1 identitat fonamental (tenim p i y)
        if not np.isnan(P) and not np.isnan(Y) and Y > 0:
            A = (P * 1000) / Y
        # 1.2 si y disponible: usar proporció y
        elif not np.isnan(Y) and mY > 0:
            A = (Y / mY) * mA
        # 1.3 si no: usar proporció p
        elif not np.isnan(P) and mP > 0:
            A = (P / mP) * mA

        row["area_harvested_ha"] = A

    #  2) imputem p (producció)
    if np.isnan(P):
        # 2.1 identitat fonamental (tenim a i y)
        if not np.isnan(A) and not np.isnan(Y) and A > 0:
            P = A * (Y / 1000)
        # 2.2 si y disponible: usar proporció y
        elif not np.isnan(Y) and mY > 0:
            P = (Y / mY) * mP
        # 2.3 si no: usar proporció a
        elif not np.isnan(A) and mA > 0:
            P = (A / mA) * mP

        row["production_kt"] = P

    #  3) imputem y (rendiment)
    if np.isnan(Y):
        # 3.1 identitat fonamental (tenim a i p)
        if not np.isnan(A) and not np.isnan(P) and A > 0:
            Y = (P / A) * 1000
        # 3.2 si a disponible: usar proporció a
        elif not np.isnan(A) and mA > 0:
            Y = (A / mA) * mY
        # 3.3 si no: usar proporció p
        elif not np.isnan(P) and mP > 0:
            Y = (P / mP) * mY

        row["yield_kg_ha"] = Y

    return row


# Apliquem imputació fila per fila
df = df.apply(impute_row, axis=1)

# Verifiquem si queden valors nuls
print("NaNs restants per columna:")
print(df[cols].isna().mean() * 100)

# Treiem les columnes de mitjanes auxiliars
df = df.drop(columns=[f"mean_{c}" for c in cols])

agri_production_final_df = df

NaNs restants per columna:
element_name
area_harvested_ha    0.0
production_kt        0.0
yield_kg_ha          0.0
dtype: float64


Finalmentm, un cop no tenim valors nuls, guardem el data set final.

In [54]:
agri_production_final_df.to_csv("../data/visualization_input_processed//agri_production_and_prices_df_processed.csv", index=False)